In [ ]:
import time, sys
from typing import Type, List, Dict, Tuple, Set
import argparse
try:
    from sklearn.externals import joblib
    from sklearn.externals.joblib import parallel_backend, Parallel, delayed
except ImportError:
    import joblib
    from joblib import parallel_backend, Parallel, delayed
    
import pandas as pd
import json, ijson
import os, sys, uuid
from pykalman import KalmanFilter
from PIL import Image
import math
import ast
import re

import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from shapely.geometry import Polygon, Point
from shapely.geometry import Polygon
from matplotlib.backends.backend_pdf import PdfPages

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from datetime import datetime
import random
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay


from collections import defaultdict
import pyarrow.parquet as pq

import pickle
from ast import literal_eval
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

import matplotlib.image as mpimg

from tqdm.notebook import tqdm, trange
from scipy.optimize import minimize
from scipy.optimize import least_squares

from os import walk
from os import listdir
from os.path import isfile, join, isdir

import scipy.optimize as opt
from shapely.geometry import Polygon
from shapely.geometry import Polygon

from PIL import Image
from matplotlib.patches import Polygon

import warnings
warnings.filterwarnings("ignore")

In [ ]:
start =time.time()
CHANNELS = [37,38,39]
N_ESTIMATORS = 100
MISSING_VALUE = -100
DEBUG_LOGGING = False
S3_CACHING_BUCKET = 'cognosos-ml-data'

In [ ]:
def parse_scan_data_woc(scan: List[Dict]) -> Dict:
    # Parse each scan to get maximum reading for each MAC address in specified channels
    readings_by_mac_addr_and_channel = defaultdict(list)
    for beacon_reading in scan:
        if beacon_reading['channel'] in CHANNELS:
            mac_addr = beacon_reading['macHex']
            readings = beacon_reading['readings']
            readings_by_mac_addr_and_channel[mac_addr] += readings
    return {mac_addr: int(max(readings)) for mac_addr, readings in readings_by_mac_addr_and_channel.items() if readings}


def process_training(data_filepath: str) -> List[Dict]:
    X = []

    # parse it incrementally
    with open(data_filepath, 'r') as f:
        # reads the JSON incrementally
        objects = ijson.items(f, 'item') 

        print('Done loading JSON incrementally')

        for scan in objects:
            
            Zone_id = str(scan['zoneId'])
            Room_name = str(scan['zoneName'])
            parent_zone_id = str(scan['parentZoneId'])
            tagId = scan['tagId']
            timestamp = scan['rxAt']
            scan_readings: List[Dict] = scan['scandata']
            
            row = parse_scan_data_woc(scan_readings) 

            row.update({
                'Zone_id': Zone_id,
                'Room_name': Room_name,
                'parent_zone_id': parent_zone_id,
                'tagId': tagId,
                'timestamp': timestamp,
            })

            if row:
                X.append(row)

    print('Done processing data')

    return X

In [ ]:
def train_variable(X_train, y_train_floor, y_train, save_models=False):
    
    floor_pipeline = Pipeline([
        ('rf', RandomForestClassifier(random_state=42))
    ])

    floor_pipeline.fit(X_train, y_train_floor)

    clf_floor = floor_pipeline.named_steps['rf']

    clf_rooms = {}

    selected_features = {}

    for floor_num, samples in X_train.groupby(y_train_floor):
        
        floor_labels = y_train[samples.index]

        non_all_neg_120_columns = samples.columns[~np.all(samples == -120, axis=0)]

        selected_samples = samples[non_all_neg_120_columns]

        classifier = RandomForestClassifier(n_estimators=200, random_state=100)

        classifier.fit(selected_samples, floor_labels)

        clf_rooms[str(floor_num)] = classifier

        selected_features[str(floor_num)] = selected_samples.columns.tolist()

    if save_models:
        model = {
        'selected_features': selected_features,
        'clf_rooms': clf_rooms,
        'clf_floor': clf_floor
        }
        joblib.dump(model, 'Hier_Features.joblib')
        
    return selected_features, clf_rooms, clf_floor

def predict_variable(X_test, clf_floor, clf_rooms, selected_features):
    
    predicted_floors = clf_floor.predict(X_test)

    predictions = []
    for floor_num, sample in zip(predicted_floors, X_test.values):
        classifier = clf_rooms[str(floor_num)]

        selected_names = selected_features[floor_num]

        selected_sample = sample[X_test.columns.isin(selected_names)].reshape(1, -1)

        predicted_room = classifier.predict(selected_sample)[0]
#         predicted_room = predicted_room.astype(str)
        predictions.append(predicted_room)

    return predictions, predicted_floors

In [ ]:
def extract_values(scan_data):

    if scan_data is None:
        return []
    return [
        {'macHex': entry['macHex'], 'channel': entry['channel'], 'readings': [entry['rssi'][0]]}
        for entry in scan_data if 'macHex' in entry and 'rssi' in entry
    ]

def parse_scan_data(scan: List[Dict]) -> Dict:

    readings_by_mac_addr_and_channel = defaultdict(list)
    for beacon_reading in scan:
        if beacon_reading['channel'] in CHANNELS:
            mac_addr = beacon_reading['macHex']
            readings = beacon_reading['readings']
            channel = beacon_reading['channel']
            readings_by_mac_addr_and_channel[f'{mac_addr}'] += readings#-{channel}
    return {mac_addr: max(readings) for mac_addr, readings in readings_by_mac_addr_and_channel.items() if len(readings) > 0 }

In [ ]:
def create_digital_twin(Anchor_point_location_file, ground_truth_file_location, map_file_location):

    anchor_df = pd.read_csv(Anchor_point_location_file)
    anchor_df["x"] = anchor_df["x"].astype(int)
    anchor_df["y"] = anchor_df["y"].astype(int)
    
    # I ADD THIS TO Ensure MAC addresses are strings and zero-padded to length 12
    anchor_df['Mac'] = anchor_df['Mac'].astype(str).str.zfill(12)
    
    macLists = anchor_df['Mac'].to_list()

    ground_truth_df = pd.read_csv(ground_truth_file_location)
    ground_truth_df["Zone_id"] = ground_truth_df["Zone_id"].astype(str)
    
    #create a empty map with 0s for future calculation
    map_ = np.zeros((65,28))

    plt.figure(figsize=(12, 6))
    
    image = Image.open(map_file_location)
    
    plt.scatter(anchor_df.x,anchor_df.y, color='blue', s=50, edgecolors='black', label='Beacons', marker='o', alpha=0.6)

#     plt.scatter(ground_truth_df["x"], ground_truth_df["y"], color='red', s=20, label='Ground Truth', marker='^')
#     for i, label in enumerate(ground_truth_df['Room_name']):  
#         plt.text(ground_truth_df['x'][i], ground_truth_df['y'][i], label, fontsize=9, color='w', ha='right', va='bottom')
    plt.imshow(image, extent=[0, 65, 0, 28], aspect='auto')

    plt.xlim(0, 65)
    plt.ylim(0, 28)

    plt.grid(True)
    plt.xticks([i for i in range(0, 65, 5)])
    plt.yticks([i for i in range(0, 28, 4)])
    plt.xlabel('x', fontsize=14)
    plt.ylabel('y', fontsize=14)
    plt.title("Beacon distribution in meters")
    plt.legend()
    plt.savefig('beacon_map_cognosos.png')

    return anchor_df, ground_truth_df, map_

In [ ]:
beacon_file = 'ground_truth/Beacon_map_cognosos_flr3.csv'
ground_truth_file = "ground_truth/Ground_truth_Mar25.csv"
map_file = 'ground_truth/Cognosos_view.png'

anchor_point_df, ground_truth_df, map_ = create_digital_twin(beacon_file, ground_truth_file, map_file)

In [ ]:
# anchor_point_df

In [ ]:
def filter_valid_features(row, df1):

    valid_features = {}
    
    for mac in df1['Mac']:
       
        if mac in row.index and isinstance(row[mac], (int, float)) and row[mac] != -100:
            valid_features[mac] = row[mac]
    
    return valid_features

def convert_coordinates(coord_str):
    if isinstance(coord_str, str):
       
        try:
            coord_str = coord_str.strip("[]")
            elements = coord_str.split()
            return [float(elem) for elem in elements] 
        except ValueError:
            pass  

        try:
            coord_str = coord_str.replace(" ", ",")
            coord_str = coord_str.replace(",,", ",")
            coord_str = coord_str.strip(',')
            return ast.literal_eval(coord_str)
        except (ValueError, SyntaxError) as e:
            print(f"Error processing coordinate string: {coord_str}")
            return None
    else:
        return coord_str

In [ ]:
def plot_predicted_all(result, ground_truth_df, map_file_location, output_file="compare_plot_MLE_NLOS.png"):
    results = []
    total_points_mle = 0
    total_points_Optimisation = 0
    total_points_fuse = 0

    total_inside_mle = 0
    total_inside_Optimisation = 0
    total_inside_fuse = 0

    merged_df = pd.merge(result, ground_truth_df, on=["Zone_id", "Room_name"], how="left")
    unique_rooms = merged_df['Room_name'].unique()

    n_rows = math.ceil(len(unique_rooms) / 2)
    fig, axes = plt.subplots(n_rows, 2, figsize=(14, 3 * n_rows))
    axes = axes.flatten()

    for i, room_name in enumerate(unique_rooms):
        room_data = merged_df[merged_df['Room_name'] == room_name]
        has_fused = 'Predicted_NLOS' in room_data.columns

        zone = room_data["Zone_id"].iloc[0]
        room_type = room_data["Room_Type"].iloc[0]
        room_box = room_data.iloc[0]

        x_coords = [room_box.get(f'x{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'x{i+1}', None))]
        y_coords = [room_box.get(f'y{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'y{i+1}', None))]

        coordinates = list(zip(x_coords, y_coords))
        polygon = Polygon(coordinates)

        if not polygon.is_valid:
            print(f"Invalid polygon for '{room_name}', attempting to fix with buffer(0).")
            polygon = polygon.buffer(0)

        # Parse MLE predictions
        x_pred_mle, y_pred_mle = [], []
        for coord in room_data["Predicted_MLE"]:
            try:
                coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                x_pred_mle.append(float(coord[0]))
                y_pred_mle.append(float(coord[1]))
            except:
                print(f"Invalid MLE coord in '{room_name}': {coord}")

        # Parse Optimisation predictions
        x_pred_Optimisation, y_pred_Optimisation = [], []
        for coord in room_data["Predicted_Optimisation"]:
            try:
                coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                x_pred_Optimisation.append(float(coord[0]))
                y_pred_Optimisation.append(float(coord[1]))
            except:
                print(f"Invalid Optimisation coord in '{room_name}': {coord}")

        # Parse Fused predictions only if available
        x_pred_fuse, y_pred_fuse = [], []
        if has_fused:
            for coord in room_data["Predicted_NLOS"]:
                try:
                    coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                    x_pred_fuse.append(float(coord[0]))
                    y_pred_fuse.append(float(coord[1]))
                except:
                    print(f"Invalid NLOS coord in '{room_name}': {coord}")

        inside_count_mle = sum(1 for x, y in zip(x_pred_mle, y_pred_mle) if Point(x, y).within(polygon))
        inside_count_Optimisation = sum(1 for x, y in zip(x_pred_Optimisation, y_pred_Optimisation) if Point(x, y).within(polygon))
        inside_count_fuse = sum(1 for x, y in zip(x_pred_fuse, y_pred_fuse) if Point(x, y).within(polygon)) if has_fused else 0

        total_points_mle += len(x_pred_mle)
        total_inside_mle += inside_count_mle

        total_points_Optimisation += len(x_pred_Optimisation)
        total_inside_Optimisation += inside_count_Optimisation

        if has_fused:
            total_points_fuse += len(x_pred_fuse)
            total_inside_fuse += inside_count_fuse

        percentage_inside_mle = (inside_count_mle / len(x_pred_mle)) * 100 if x_pred_mle else 0
        percentage_inside_Optimisation = (inside_count_Optimisation / len(x_pred_Optimisation)) * 100 if x_pred_Optimisation else 0
        percentage_inside_fuse = (inside_count_fuse / len(x_pred_fuse)) * 100 if has_fused and x_pred_fuse else 0

        results.append({
            "Zone_id": zone,
            'Room_name': room_name,
            "Room_Type": room_type,
            'MLE_Accuracy': percentage_inside_mle,
            'Optimisation_Accuracy': percentage_inside_Optimisation,
            'NLOS_Accuracy': percentage_inside_fuse if has_fused else None,
            'MLE_Inside_Points': inside_count_mle,
            'Optimisation_Inside_Points': inside_count_Optimisation,
            'NLOS_Inside_Points': inside_count_fuse if has_fused else None,
            'Total_Points': len(x_pred_mle),
        })

        # Plot
        ax = axes[i]
        image = mpimg.imread(map_file_location)
        ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')
        ax.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], 'r-', label='Room Boundary')
        ax.scatter(x_pred_mle, y_pred_mle, color='blue', s=8, label='MLE')
        ax.scatter(x_pred_Optimisation, y_pred_Optimisation, color='green', s=8, label='Optimisation')
        if has_fused:
            ax.scatter(x_pred_fuse, y_pred_fuse, color='red', s=8, label='NLOS')

        ax.set_xlim([0, 65])
        ax.set_ylim([0, 28])
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        title_str = f"{room_name} - MLE: {percentage_inside_mle:.1f}%, Optimisation: {percentage_inside_Optimisation:.1f}%"
        if has_fused:
            title_str += f", NLOS: {percentage_inside_fuse:.1f}%"
        ax.set_title(title_str)
        ax.legend(loc='lower left', bbox_to_anchor=(0, 0), ncol=2)

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    overall_mle_accuracy = (total_inside_mle / total_points_mle) * 100 if total_points_mle > 0 else 0
    overall_Optimisation_accuracy = (total_inside_Optimisation / total_points_Optimisation) * 100 if total_points_Optimisation > 0 else 0
    overall_fuse_accuracy = (total_inside_fuse / total_points_fuse) * 100 if total_points_fuse > 0 else 0

    print(f"\nOverall MLE Accuracy: {overall_mle_accuracy:.2f}%")
    print(f"Overall Optimisation Accuracy: {overall_Optimisation_accuracy:.2f}%")
    if total_points_fuse > 0:
        print(f"Overall NLOS Accuracy: {overall_fuse_accuracy:.2f}%")

    accuracy_df = pd.DataFrame(results)
    plt.tight_layout()
    plt.savefig(output_file, format="png")
    plt.show()

    return accuracy_df

In [ ]:
def read_MLE_data_survey_portal(df, ground_truth_df, anchor_point_df, export_unheard=False, export_path="unheard_anchor_points.csv"):

    # I ADD THIS TO Ensure MAC addresses are strings and zero-padded to length 12
    anchor_point_df['Mac'] = anchor_point_df['Mac'].astype(str).str.zfill(12)

    data_set_df = pd.DataFrame()
    merged_df = pd.merge(df, ground_truth_df, on=["Zone_id", 'Room_name'], how='inner').drop(['parent_zone_id'], axis=1)
    zones = df['Zone_id']
    heard_anchor_points = []

    for mac_addr in anchor_point_df['Mac']:
        if mac_addr in merged_df.columns:
            data_set_df[mac_addr] = merged_df[mac_addr]
            heard_anchor_points.append(mac_addr)

    heard_anchor_point_df = anchor_point_df[anchor_point_df['Mac'].isin(heard_anchor_points)].reset_index(drop=True)
    unheard_anchor_point_df = anchor_point_df[~anchor_point_df['Mac'].isin(heard_anchor_points)].reset_index(drop=True)

    heard_anchor_points_coord = heard_anchor_point_df[['x', 'y']].values

    data_set_df["Zone_id"] = merged_df["Zone_id"]
    data_set_df["Room_name"] = merged_df["Room_name"]
    data_set_df["tagId"] = merged_df["tagId"]
    data_set_df["timestamp"] = merged_df["timestamp"]
    if "channel" in merged_df.columns:
        data_set_df["channel"] = merged_df["channel"]
    
    data_set_df["x"] = merged_df["x"]
    data_set_df["y"] = merged_df["y"]

    return data_set_df, heard_anchor_points_coord, unheard_anchor_point_df

In [ ]:
filepath = "data_walking_Stats.json"
filename = os.path.basename(filepath)

X1 = process_training(filepath)
df1 = pd.DataFrame(X1)
df1 = df1.fillna(MISSING_VALUE)  

float_cols = df1.select_dtypes(include=['float']).columns
df1[float_cols] = df1[float_cols].astype(np.int8)
df1['timestamp'] = pd.to_datetime(df1['timestamp'], utc=True, errors='coerce')
df1 = df1.sort_values(by='timestamp')

ordered_columns = ['timestamp', 'tagId', 'Zone_id', 'Room_name', "parent_zone_id"]

columns = [col for col in anchor_point_df.Mac.unique().tolist() if col not in ordered_columns]
new_column_order = columns + ordered_columns
df1 = df1.reindex(columns=new_column_order)

df1 = df1.reset_index(drop=True)
df1['Room_name'] = df1['Room_name'].str.split('-').str[-1].str.strip()

# Fix specific zone_id
df1.loc[df1['Zone_id'] == "30598", 'Zone_id'] = "30539"

# # Conditional exclusion based on filename
# if filename == "data_duress_access_Jun23.json":
#     rooms_to_exclude = ["Tech Office 2"]  # this room has issue with data as very low beacon signal, beacon count
#     df1 = df1[~df1['Room_name'].isin(rooms_to_exclude)]

# Beacon processing
beacon_cols = [col for col in df1.columns if str(col).startswith('0')]
df1 = df1.fillna(MISSING_VALUE)
df1['beacon_count'] = (df1[beacon_cols] != -100).sum(axis=1)
# df1= df1[df1.Zone_id.isin(zones)]
print(df1.shape)

df1 = df1[df1['beacon_count'] >= 5]
print(df1.shape)

In [ ]:
df1['beacon_count'].max(), df1['beacon_count'].min()

In [ ]:
# df = pd.concat([df1, df2], axis=0, ignore_index=True, sort=False)
# df.shape

In [ ]:
df1.Room_name.nunique()

In [ ]:
# df1= df.copy()

In [ ]:
# df1.to_csv("data/Duress/data_duress_edgecase_combine.csv", index= False)

## REMOVE ALL ROWS < -90

In [ ]:
beacon_cols = [col for col in df1.columns if str(col).startswith('0')]
rows_all_below_90 = df1[beacon_cols].lt(-99).all(axis=1)
df1 = df1[~rows_all_below_90]
df1.shape

In [ ]:
df1.Room_name.nunique()

In [ ]:
df1.tagId.unique()

ankle: '7002352', '7002100', '7002065, '7002074'

pocket: '7000588', '7000245'

front: '7003640', '7003999', '7004370', '7004333'

back: '7004162', '7004545', '7004268', '7004050'

### Check if dataset have enoguh beacon heard >=-90, SELECT ONLY the number of strong features >=5

In [ ]:
df2 = pd.merge(df1.drop(columns=["parent_zone_id", "beacon_count"]), \
                                ground_truth_df[['Zone_id','x', 'y']], on=["Zone_id"], how='left')
rssi_cols = [col for col in df2.columns if col.startswith('0')]

# Create a new column counting RSSIs >= -90
df2['num_strong_features'] = (df2[rssi_cols] >= -95).sum(axis=1)

df2.shape

In [ ]:
data_set_df = df2[df2['num_strong_features'] >= 5].copy()

data_set_df=data_set_df.drop(columns='num_strong_features').reset_index(drop=True)
data_set_df.shape

In [ ]:
# df2.iloc[[2885]].describe().T.sort_values(by="max", ascending= False)

# Take 10% sample

In [ ]:
# data_set_df = data_set_df_all.groupby('Zone_id', group_keys=False).\
#         apply(lambda x: x.sample(frac=1, random_state=42))
# data_set_df.shape

# *** MAKE SURE THAT ALL BEACONS ARE BEING HEARD

In [ ]:
## Check if any ionstalled beacon not heard by the tags
# unheard_anchor_point_df

In [ ]:
data_set_df.Room_name.nunique()

In [ ]:
anchor_point_df.head()

In [ ]:
anchor_point_df.shape

In [ ]:
len(data_set_df.columns.intersection(anchor_point_df.Mac))

In [ ]:
data_set_df.shape

# NLOS: Fused of Opt and MLE

In [ ]:
def euclidean_dist(point, points, height_diff=1.5):
    return np.sqrt(np.sum((points - point) ** 2, axis=1) + height_diff ** 2)

def generate_grid(center, resolution=5, radius=4):
    step = 1 / resolution
    x_vals = np.arange(center[0] - radius, center[0] + radius + step, step)
    y_vals = np.arange(center[1] - radius, center[1] + radius + step, step)
    xv, yv = np.meshgrid(x_vals, y_vals)
    return np.stack([xv.ravel(), yv.ravel()], axis=1)

def trilateration(coords, distances):
    def fun(x, coords, distances):
        weights = 1 / (distances + 1e-5)
        return weights * (np.linalg.norm(coords - x, axis=1) - distances)
    x0 = np.mean(coords, axis=0)
    result = least_squares(fun, x0, args=(coords, distances))
    return result.x if result.success else x0

def rssi_to_distance(rssi, A=-65, n=3.5, scale=0.8):
    return scale * np.exp((A - rssi) / (10 * n))

def filter_rssi(row, beacon_positions, rssi_threshold=-90):
    return {
        mac: row[mac]
        for mac in beacon_positions.keys()
        if mac in row.index and isinstance(row[mac], (int, float)) and row[mac] > rssi_threshold
    }

def localization_error(tag_position, beacons, distances):
    estimated_distances = np.linalg.norm(beacons - tag_position, axis=1)
    sigma = np.std(distances) + 1e-3
    weights = np.exp(- (distances ** 2) / (2 * sigma ** 2))
    return np.sum(weights * (estimated_distances - distances) ** 2)

def generate_expansion_area(initial_guess, std_dev=0.5, radius=0.8, num_points=500, range_box=2):
    num_gauss = int(num_points * 0.4)
    num_box = int(num_points * 0.3)
    num_circle = num_points - num_gauss - num_box
    box_points = np.random.uniform(-range_box, range_box, size=(num_box, 2)) + initial_guess
    gauss_points = np.random.normal(0, std_dev, size=(num_gauss, 2)) + initial_guess
    r = radius * np.sqrt(np.random.uniform(0, 1, num_circle))
    theta = np.random.uniform(0, 2 * np.pi, num_circle)
    circ_points = np.column_stack((initial_guess[0] + r * np.cos(theta), initial_guess[1] + r * np.sin(theta)))
    return np.vstack((gauss_points, box_points, circ_points))

def calculate_total_error_to_all_beacons(best_position, beacon_coords):
    distances = np.linalg.norm(beacon_coords - best_position, axis=1)
    return np.sum(distances)

def compute_likelihood_weighted(grid_coords, anchor_coords, rssi_values, T, n, sigma_noise=4, anchor_weights=None):
    if anchor_weights is None:
        anchor_weights = np.ones_like(rssi_values)
    diff = grid_coords[:, None, :] - anchor_coords[None, :, :]
    dists = np.sqrt(np.sum(diff ** 2, axis=2) + 1.5**2)
    pred_rssi = T - 10 * n * np.log10(dists + 1e-5)
    residuals = pred_rssi - rssi_values
    weighted_residuals = (residuals / sigma_noise)**2 * anchor_weights
    likelihood = np.exp(-0.5 * weighted_residuals)
    return np.prod(likelihood, axis=1)

def find_mle_params(P_j, d_ij, init_guess=[-45, 3]):  # Chneg from -65
    def squared_error(params, dists, rssi):
        T_i, n_p = params
        valid_mask = rssi != -100
        pred_rssi = T_i - 10 * n_p * np.log10(dists + 1e-5)
        return np.sum((pred_rssi[valid_mask] - rssi[valid_mask]) ** 2)
    bounds = [(-100, -30), (2, 6)]
    result = minimize(squared_error, init_guess, args=(d_ij, P_j),
                      method='L-BFGS-B', bounds=bounds)
    return result.x if result.success else init_guess

# Fused Localization Function
# -------------------------------
def fused_localization_ref(data_df, anchor_point_df,
                       sigma_noise=4, coarse_res=2, fine_res=5, fine_radius=3,
                       rssi_threshold=-95, strong_rssi_threshold=-75,
                       top_k_anchors=5, roi_margin=8, top_coarse_points=200, topN_ratio=0.05,
                       map_x_bounds=(0, 65), map_y_bounds=(0, 28),
                       epsilon=1e-12,
                       min_beacons_opt=3, max_beacons_opt=20,
                       expansion_radius=1, expansion_points=100, top_expansion_points=15,
                       enable_refinement=True):
    
    results = []
    beacon_positions = anchor_point_df[["x","y","Mac"]].set_index("Mac")[["x","y"]].to_dict(orient="index")
    has_timestamp = 'timestamp' in data_df.columns

    for idx, row in tqdm(data_df.iterrows(), total=len(data_df)):
        # ---------------------------
        # Extract RSSI
        # ---------------------------
        rssis = row.drop(['Zone_id','Room_name','x','y','tagId','timestamp'], errors='ignore').values.astype(float)
        anchor_coords = anchor_point_df[['x','y']].values

        # ---------------------------
        # MLE
        # ---------------------------
        mask_mle = rssis > rssi_threshold
        signal_strengths = rssis[mask_mle]
        dp_coords = anchor_coords[mask_mle]

        if len(signal_strengths) < 1:
            signal_strengths = rssis
            dp_coords = anchor_coords

        strong_mask = signal_strengths > strong_rssi_threshold
        if np.sum(strong_mask) < 2:
            dp_coords_selected = dp_coords
            signal_strengths_selected = signal_strengths
        else:
            dp_coords_selected = dp_coords[strong_mask]
            signal_strengths_selected = signal_strengths[strong_mask]

        min_rssi, max_rssi = np.min(signal_strengths_selected), np.max(signal_strengths_selected)
        anchor_weights = (signal_strengths_selected - min_rssi + 1) / (max_rssi - min_rssi + 1e-5)
        sorted_idx = np.argsort(-signal_strengths_selected)
        top_k = min(top_k_anchors, len(sorted_idx))
        top_coords = dp_coords_selected[sorted_idx[:top_k]]

        x_min, y_min = np.min(top_coords, axis=0)
        x_max, y_max = np.max(top_coords, axis=0)
        x_min = max(x_min - roi_margin, map_x_bounds[0])
        x_max = min(x_max + roi_margin, map_x_bounds[1])
        y_min = max(y_min - roi_margin, map_y_bounds[0])
        y_max = min(y_max + roi_margin, map_y_bounds[1])

        coarse_grid = np.stack(np.meshgrid(np.arange(x_min, x_max, 1/coarse_res),
                                           np.arange(y_min, y_max, 1/coarse_res)), axis=-1).reshape(-1,2)

        strongest_coord = top_coords[0]
        dists_for_fit = euclidean_dist(strongest_coord, dp_coords_selected)
        T_global, n_global = find_mle_params(signal_strengths_selected, dists_for_fit)

        coarse_likelihoods = compute_likelihood_weighted(
            coarse_grid, dp_coords_selected, signal_strengths_selected,
            T_global, n_global, sigma_noise, anchor_weights
        )

        top_indices = np.argpartition(coarse_likelihoods, -top_coarse_points)[-top_coarse_points:]
        top_candidates = coarse_grid[top_indices]

        fine_candidates, fine_likelihoods = [], []
        for center in top_candidates:
            fine_grid = generate_grid(center, resolution=fine_res, radius=fine_radius)
            likelihoods_fine = compute_likelihood_weighted(
                fine_grid, dp_coords_selected, signal_strengths_selected,
                T_global, n_global, sigma_noise, anchor_weights
            )
            fine_candidates.append(fine_grid)
            fine_likelihoods.append(likelihoods_fine)

        fine_candidates = np.vstack(fine_candidates)
        fine_likelihoods = np.hstack(fine_likelihoods)
        fine_likelihoods += epsilon
        fine_likelihoods /= np.sum(fine_likelihoods)

        N = max(1, min(100, int(topN_ratio * len(fine_candidates))))
        top_idx = np.argpartition(fine_likelihoods, -N)[-N:]
        top_points = fine_candidates[top_idx]
        top_weights = fine_likelihoods[top_idx]
        top_weights /= np.sum(top_weights)
        pred_mle = np.average(top_points, axis=0, weights=top_weights)
        conf_mle = np.max(top_weights)

        # ---------------------------
        # Optimization
        # ---------------------------
        rssi_values_opt = dict(sorted(filter_rssi(row, beacon_positions, rssi_threshold).items(), key=lambda x: x[1], reverse=True))
        if len(rssi_values_opt) < min_beacons_opt:
            beacon_coords_opt = anchor_coords
            distances_opt = np.ones(anchor_coords.shape[0])
        else:
            beacon_coords_opt = np.array([list(beacon_positions[b].values()) for b in rssi_values_opt.keys()])
            distances_opt = np.array([rssi_to_distance(rssi) for rssi in rssi_values_opt.values()])

        close_beacons = beacon_coords_opt[distances_opt <= 1]
        if len(close_beacons) > 0:
            initial_guess_opt = close_beacons[np.argmin(distances_opt[distances_opt <= 1])]
        else:
            top_n = min(5, len(distances_opt))
            initial_guess_opt = trilateration(beacon_coords_opt[:top_n], distances_opt[:top_n])

        best_positions_opt, total_errors_opt, num_beacons_used = [], [], []
        for num_beacons in range(min_beacons_opt, min(len(distances_opt), max_beacons_opt)+1):
            top_n_coords = beacon_coords_opt[:num_beacons]
            top_n_dists = distances_opt[:num_beacons]

            expansion_area = generate_expansion_area(initial_guess_opt, radius=expansion_radius, num_points=expansion_points)
            quick_errors = np.array([localization_error(p, top_n_coords, top_n_dists) for p in expansion_area])
            filtered_expansion_area = expansion_area[np.argsort(quick_errors)[:top_expansion_points]]

            best_err, best_pos = float("inf"), None
            for point in filtered_expansion_area:
                res = minimize(localization_error, point, args=(top_n_coords, top_n_dists), method='L-BFGS-B', options={'maxiter':100})
                if res.success:
                    est_pos = res.x
                    total_err = np.sum(np.linalg.norm(top_n_coords - est_pos, axis=1))
                    if total_err < best_err:
                        best_err = total_err
                        best_pos = est_pos
            best_positions_opt.append(best_pos)
            total_errors_opt.append(best_err)
            num_beacons_used.append(num_beacons)

        # Final selection based on total error
        best_idx_opt = np.argmin([calculate_total_error_to_all_beacons(p, beacon_coords_opt) for p in best_positions_opt])
        pred_opt = best_positions_opt[best_idx_opt]
        conf_opt = 1 / (1 + total_errors_opt[best_idx_opt])

        # ---------------------------
        # Final Refinement (matches original code)
        # ---------------------------
        pre_refined_pos = pred_opt.copy()
        if enable_refinement:
            refinement_rssi_threshold = -75
            strong_rssi_indices = [i for i, rssi in enumerate(rssi_values_opt.values()) if rssi > refinement_rssi_threshold]

            if len(strong_rssi_indices) >= 3:
                filtered_coords = beacon_coords_opt[strong_rssi_indices]
                filtered_distances = distances_opt[strong_rssi_indices]

                result = minimize(
                    localization_error,
                    pred_opt,
                    args=(filtered_coords, filtered_distances),
                    method='L-BFGS-B',
                    options={'maxiter': 100, 'gtol': 1e-8, 'disp': False}
                )
                if result.success:
                    pred_opt = result.x

        refinement_shift = np.linalg.norm(pred_opt - pre_refined_pos)

        # ---------------------------
        # Fused Results
        # ---------------------------
        alpha_dynamic = conf_mle / (conf_mle + conf_opt)
        pred_fused_fixed = 0.5 * pred_mle + 0.5 * pred_opt
        pred_fused_dynamic = alpha_dynamic * pred_mle + (1-alpha_dynamic) * pred_opt

        results.append({
            'original_index': idx,
            'Zone_id': row.get('Zone_id', np.nan),
            'Room_name': row.get('Room_name', np.nan),
            'Tag_id': row.get('tagId', np.nan),
            'timestamp': row.get('timestamp', np.nan),
            'Predicted_MLE': pred_mle,
            'Predicted_Optimisation': pred_opt,
            'Predicted_NLOS': pred_fused_fixed,
            'Predicted_NLOS_Dynamic': pred_fused_dynamic,
            'MLE_Confidence': conf_mle,
            'Opt_Confidence': conf_opt,
            'Ground_Truth': np.array([row['x'], row['y']])
        })

    return pd.DataFrame(results)


In [ ]:
filename

In [ ]:
start_time = time.perf_counter() 

result= fused_localization_ref(data_set_df, anchor_point_df)

save_folder = "Result_Duress"
save_name = f"{filename.replace('.json', '_NLOS_Real_Time.csv')}" 
save_path = os.path.join(save_folder, save_name)

result.to_csv(save_path, index=False)


end_time = time.perf_counter() 

total_time = end_time - start_time
avg_time_per_row = total_time / len(data_set_df)
print(avg_time_per_row)

In [ ]:
result

In [ ]:
# result= results_df.copy()
# result= pd.read_csv("Result_Duress/data_duress_access_Jun23_Fused.csv")
# result['Predicted_MLE'] = result['Predicted_MLE'].apply(convert_coordinates)
# result['Ground_Truth'] = result['Ground_Truth'].apply(convert_coordinates)


# result["Predicted_Optimisation"]= result['Predicted_Optimisation'].apply(convert_coordinates)
# result["Predicted_Fused"]= result['Predicted_Fused'].apply(convert_coordinates)
# result["Fused_Alpha_0.5"]= result['Fused_Alpha_0.5'].apply(convert_coordinates)

# result.Zone_id= result.Zone_id.astype(str)
# # result.Tag_id= result.Tag_id.astype(str)
# result= result[result.Room_name !='Womens Restroom']
# result.head(1)

In [ ]:
from shapely.geometry import Polygon

def plot_predicted_fused_dynamic(result_df, ground_truth_df, map_file_location,
                                 fused_cols=['Predicted_NLOS_Dynamic', 'Predicted_NLOS'],
                                 output_file="compare_plot_MLE_Optim_Fused.png"):
    """
    Plot predicted locations from MLE, Optimisation, and fused methods, showing inside-room accuracy.
    Computes overall accuracy and returns per-room statistics including inside-point counts and total points.
    Also provides room-type aggregated accuracy.
    """
    results = []

    merged_df = pd.merge(result_df, ground_truth_df, on=["Zone_id", "Room_name"], how="left")
    unique_rooms = merged_df['Room_name'].unique()

    n_rows = math.ceil(len(unique_rooms) / 2)
    fig, axes = plt.subplots(n_rows, 2, figsize=(14, 3 * n_rows))
    axes = axes.flatten()

    total_inside = {"MLE": 0, "Optimisation": 0}
    total_inside.update({col: 0 for col in fused_cols})
    total_points = 0  # only one total points count

    for i, room_name in enumerate(unique_rooms):
        room_data = merged_df[merged_df['Room_name'] == room_name]
        zone = room_data["Zone_id"].iloc[0]
        room_type = room_data["Room_Type"].iloc[0]
        room_box = room_data.iloc[0]

        # Room polygon
        x_coords = [room_box.get(f'x{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'x{i+1}', None))]
        y_coords = [room_box.get(f'y{i+1}', None) for i in range(8) if pd.notnull(room_box.get(f'y{i+1}', None))]
        coordinates = list(zip(x_coords, y_coords))
        polygon = Polygon(coordinates)
        if not polygon.is_valid:
            polygon = polygon.buffer(0)

        # Helper function to parse coordinates
        def parse_coords(col_name):
            x_list, y_list = [], []
            for coord in room_data[col_name]:
                try:
                    coord = ast.literal_eval(coord) if isinstance(coord, str) else coord
                    x_list.append(float(coord[0]))
                    y_list.append(float(coord[1]))
                except:
                    pass
            return x_list, y_list

        # Predictions
        x_mle, y_mle = parse_coords("Predicted_MLE")
        x_opt, y_opt = parse_coords("Predicted_Optimisation")
        fused_data = {col: parse_coords(col) for col in fused_cols if col in room_data.columns}

        # Count points inside polygon
        def count_inside(x_list, y_list):
            return sum(1 for x, y in zip(x_list, y_list) if Point(x, y).within(polygon))

        inside_mle = count_inside(x_mle, y_mle)
        inside_opt = count_inside(x_opt, y_opt)
        inside_fused = {k: count_inside(*v) for k, v in fused_data.items()}

        # Update totals
        total_inside["MLE"] += inside_mle
        total_inside["Optimisation"] += inside_opt
        for k, v in fused_data.items():
            total_inside[k] += inside_fused[k]
        total_points += len(x_mle)  # same for all methods

        # Save per-room results
        results.append({
            "Zone_id": zone,
            "Room_name": room_name,
            "Room_Type": room_type,
            "MLE_Accuracy": inside_mle / max(len(x_mle), 1) * 100,
            "Optimisation_Accuracy": inside_opt / max(len(x_opt), 1) * 100,
            **{f"{k}_Accuracy": inside_fused[k] / max(len(fused_data[k][0]), 1) * 100 for k in fused_data},
            "MLE_Inside_Points": inside_mle,
            "Optimisation_Inside_Points": inside_opt,
            **{f"{k}_Inside_Points": inside_fused[k] for k in fused_data},
            "Total_Points": len(x_mle)
        })

        # Plotting
        ax = axes[i]
        image = mpimg.imread(map_file_location)
        ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')
        ax.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], 'r-', label='Room Boundary')
        ax.scatter(x_mle, y_mle, color='blue', s=8, label='MLE')
        ax.scatter(x_opt, y_opt, color='green', s=8, label='Optimisation')
        colors = ['orange', 'purple', 'red', 'cyan']
        for j, (fcol, (x_f, y_f)) in enumerate(fused_data.items()):
            ax.scatter(x_f, y_f, color=colors[j % len(colors)], s=8, label=f"{fcol}")

        ax.set_xlim([0, 65])
        ax.set_ylim([0, 28])
        
        # Build title using percentage accuracy instead of counts
        title_str = (
            f"{room_name} - "
            f"MLE: {inside_mle / max(len(x_mle), 1) * 100:.1f}%, "
            f"Opt: {inside_opt / max(len(x_opt), 1) * 100:.1f}%"
        )

        for fcol, (x_f, y_f) in fused_data.items():
            acc = inside_fused[fcol] / max(len(x_f), 1) * 100
            clean_name = fcol.replace("Predicted_", "")  # <--- removes the prefix
            title_str += f", {clean_name}: {acc:.1f}%"


            
        ax.set_title(title_str)
        ax.legend(loc='lower left', bbox_to_anchor=(0, 0), ncol=2)

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    accuracy_df = pd.DataFrame(results)

    # --- Overall accuracy ---
    print("\n=== Overall Accuracy ===")
    for method in total_inside.keys():
        overall = total_inside[method] / max(total_points, 1) * 100
        print(f"{method}: {overall:.2f}%")

    # --- Room-type aggregated accuracy ---
    room_type_stats = accuracy_df.groupby('Room_Type').agg({
        'MLE_Inside_Points': 'sum',
        'Optimisation_Inside_Points': 'sum',
        **{f"{col}_Inside_Points": 'sum' for col in fused_cols},
        'Total_Points': 'sum'
    })

    for method in ['MLE', 'Optimisation'] + fused_cols:
        room_type_stats[f"{method}_Accuracy"] = room_type_stats[f"{method}_Inside_Points"] / room_type_stats['Total_Points'] * 100

    plt.tight_layout()
    plt.savefig(output_file, format="png")
    plt.show()

    return accuracy_df

from matplotlib.patches import Patch
from shapely.geometry import Polygon

In [ ]:
accuracy_df = plot_predicted_fused_dynamic(
    result_df=result,
    ground_truth_df=ground_truth_df,
    map_file_location= map_file,
    fused_cols=['Predicted_NLOS_Dynamic', 'Predicted_NLOS'],
#     output_file="compare_fused_results.png"
)

In [ ]:
# Group by Room_Type and compute weighted (point-based) accuracy
weighted_grouped = (
    accuracy_df
    .groupby("Room_Type")
    .apply(lambda g: pd.Series({
        "MLE_Accuracy": (g["MLE_Inside_Points"].sum() / g["Total_Points"].sum()) * 100,
        "Optimisation_Accuracy": (g["Optimisation_Inside_Points"].sum() / g["Total_Points"].sum()) * 100,
        "NLOS_Accuracy": (g["Predicted_NLOS_Inside_Points"].sum() / g["Total_Points"].sum()) * 100
    }))
)

# Calculate overall accuracy (also weighted)
overall = pd.DataFrame([{
    "MLE_Accuracy": (accuracy_df["MLE_Inside_Points"].sum() / accuracy_df["Total_Points"].sum()) * 100,
    "Optimisation_Accuracy": (accuracy_df["Optimisation_Inside_Points"].sum() / accuracy_df["Total_Points"].sum()) * 100,
    "NLOS_Accuracy": (accuracy_df["Predicted_NLOS_Inside_Points"].sum() / accuracy_df["Total_Points"].sum()) * 100
}], index=["Overall"])

# Combine results
summary_df = pd.concat([overall, weighted_grouped]).rename(index={"Open": "Open Space"})
summary_df.to_csv("Result_Duress/temp_result.csv", index= False)
summary_df

In [ ]:
from matplotlib.patches import Patch

from matplotlib.patches import Patch

def plot_accuracy_per_room(
    accuracy_df,
    ground_truth_df,
    map_file_location,
    colors=("green", "blue", "purple"),
    labels=("MLE", "Optimisation", "Fused"),
    title_text="Room-wise Accuracy",
    output_file=None
):
    """
    Plot room-wise numeric accuracies for three models in different colors.
    First line: MLE/Optimisation (no spaces)
    Second line: Fused
    """

    # Merge with ground truth polygons
    merged_df = pd.merge(accuracy_df, ground_truth_df, on=["Zone_id", "Room_name"], how="left")

    fig, ax = plt.subplots(figsize=(16, 8))
    image = mpimg.imread(map_file_location)
    ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto', zorder=0)

    for _, row in merged_df.iterrows():
        # Polygon coordinates
        x_coords = [row.get(f"x{i+1}", None) for i in range(8) if pd.notnull(row.get(f"x{i+1}", None))]
        y_coords = [row.get(f"y{i+1}", None) for i in range(8) if pd.notnull(row.get(f"y{i+1}", None))]
        if not x_coords or not y_coords:
            continue

        polygon = Polygon(list(zip(x_coords, y_coords)))
        if not polygon.is_valid:
            polygon = polygon.buffer(0)

        # Draw polygon outline
        ax.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], 'k-', lw=1, zorder=2)

        # Accuracy values
        acc1 = int(row.get(f"{labels[0]}_Accuracy", 0))
        acc2 = int(row.get(f"{labels[1]}_Accuracy", 0))
        acc3 = int(row.get(f"{labels[2]}_Accuracy", 0))

        centroid = polygon.centroid

        # Line 1: MLE / Optimisation
        ax.text(
            centroid.x, centroid.y + 0.15,
            f"{acc1}/{acc2}",
            color="black", fontsize=10, ha="center", va="center", fontweight="bold"
        )

        # Individual colors
        ax.text(centroid.x - 0.55, centroid.y + 0.15, f"{acc1}", color=colors[0], fontsize=10,
                ha="center", va="center", zorder=4, fontweight="bold")
        ax.text(centroid.x, centroid.y + 0.15, "/", color="black", fontsize=10,
                ha="center", va="center", zorder=4)
        ax.text(centroid.x + 0.55, centroid.y + 0.15, f"{acc2}", color=colors[1], fontsize=10,
                ha="center", va="center", zorder=4, fontweight="bold")

        # Line 2: Fused
        ax.text(
            centroid.x, centroid.y - 0.45,
            f"{acc3}",
            color=colors[2], fontsize=10, ha="center", va="center", zorder=3, fontweight="bold"
        )

    # Automatically scale axes
    all_x = pd.concat([ground_truth_df[f"x{i+1}"] for i in range(8)], axis=0, ignore_index=True).dropna()
    all_y = pd.concat([ground_truth_df[f"y{i+1}"] for i in range(8)], axis=0, ignore_index=True).dropna()
    ax.set_xlim([all_x.min() - 1, all_x.max() + 1])
    ax.set_ylim([all_y.min() - 1, all_y.max() + 1])

    # ✅ Weighted overall accuracies
    total_points = accuracy_df["Total_Points"].sum()
    overall_acc = []
    for label in labels:
        inside_col = f"{label}_Inside_Points"
        if inside_col in accuracy_df:
            overall = (accuracy_df[inside_col].sum() / total_points) * 100
            overall_acc.append(overall)
        else:
            overall_acc.append(0)

    overall_text = " | ".join([f"{label}: {val:.1f}%" for label, val in zip(labels, overall_acc)])
    ax.set_title(f"{title_text}\nOverall Accuracy: {overall_text}", fontsize=15, fontweight="bold")

    # Legend
    legend_handles = [Patch(color=color, label=label) for color, label in zip(colors, labels)]
    ax.legend(handles=legend_handles, loc="lower left")

    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    plt.tight_layout()

    if output_file:
        plt.savefig(output_file, dpi=150)

In [ ]:
filename

In [ ]:
plot_accuracy_per_room(
    accuracy_df=accuracy_df,
    ground_truth_df=ground_truth_df,
    map_file_location=map_file,
    colors=("green", "blue", "purple"),
    labels=("MLE", "Optimisation", "Predicted_Fused"),
    title_text="DURESS_Normal Case_Single Data Packet",
#     output_file="Result_Duress/compare_plot_Fused_data_duress_access_Jun23.png"
)

In [ ]:
def compute_accuracy_per_tag(
    result_df,
    ground_truth_df,
    fused_cols=['Predicted_NLOS', 'Predicted_NLOS_Dynamic']
):
    """
    Computes accuracy per Tag_id for MLE, Optimisation, and fused models.
    Returns a DataFrame containing inside counts, total points, and accuracy (%)
    for each tag in each room.
    """

    # Merge predictions with polygon definitions
    merged = pd.merge(result_df, ground_truth_df,
                      on=["Zone_id", "Room_name"], how="left")

    results = []

    # Iterate room by room
    for (zone_id, room_name), room_df in merged.groupby(["Zone_id", "Room_name"]):

        # Extract polygon
        box = room_df.iloc[0]
        x_coords = [box.get(f"x{i+1}") for i in range(8) if pd.notnull(box.get(f"x{i+1}"))]
        y_coords = [box.get(f"y{i+1}") for i in range(8) if pd.notnull(box.get(f"y{i+1}"))]
        polygon = Polygon(list(zip(x_coords, y_coords))).buffer(0)

        # Group by tag
        for tag_id, tag_df in room_df.groupby("Tag_id"):

            # Helper to parse prediction coordinates
            def parse_coords(series):
                xs, ys = [], []
                for v in series:
                    try:
                        v = ast.literal_eval(v) if isinstance(v, str) else v
                        xs.append(float(v[0]))
                        ys.append(float(v[1]))
                    except:
                        pass
                return xs, ys

            # --- Base models ---
            x_mle, y_mle = parse_coords(tag_df["Predicted_MLE"])
            x_opt, y_opt = parse_coords(tag_df["Predicted_Optimisation"])

            def inside_count(xs, ys):
                return sum(Point(x, y).within(polygon) for x, y in zip(xs, ys))

            mle_inside = inside_count(x_mle, y_mle)
            opt_inside = inside_count(x_opt, y_opt)

            # Save base results
            result_row = {
                "Zone_id": zone_id,
                "Room_name": room_name,
                "Tag_id": tag_id,
                "MLE_Inside": mle_inside,
                "Optimisation_Inside": opt_inside,
                "Total_Points": len(x_mle),
                "MLE_Accuracy": 100 * mle_inside / max(len(x_mle), 1),
                "Optimisation_Accuracy": 100 * opt_inside / max(len(x_opt), 1),
            }

            # --- Fused Methods ---
            for col in fused_cols:
                if col in tag_df.columns:
                    xf, yf = parse_coords(tag_df[col])
                    inside_f = inside_count(xf, yf)
                    result_row[f"{col}_Inside"] = inside_f
                    result_row[f"{col}_Accuracy"] = 100 * inside_f / max(len(xf), 1)

            results.append(result_row)

    return pd.DataFrame(results)

def compute_overall_accuracy_per_tag(tag_accuracy_df):
    """
    Compute overall accuracy per Tag_id by summing inside points and total points,
    then dividing to get accuracy (%) for each method.
    """

    # Columns for models
    base_models = ["MLE", "Optimisation"]
    fused_models = [col.replace("_Inside", "") for col in tag_accuracy_df.columns if col.endswith("_Inside") 
                    and col not in ["MLE_Inside", "Optimisation_Inside"]]

    all_models = base_models + fused_models

    # Group by Tag_id
    grouped = tag_accuracy_df.groupby("Tag_id").agg(
        {f"{m}_Inside": "sum" for m in all_models} | {"Total_Points": "sum"}
    ).reset_index()

    # Compute overall accuracy
    for m in all_models:
        grouped[f"{m}_Overall_Accuracy"] = grouped[f"{m}_Inside"] / grouped["Total_Points"] * 100

    return grouped


In [ ]:
tag_accuracy_df = compute_accuracy_per_tag(result, ground_truth_df)
overall_df = compute_overall_accuracy_per_tag(tag_accuracy_df)
overall_df[["Tag_id", "MLE_Overall_Accuracy", "Optimisation_Overall_Accuracy", "Predicted_NLOS_Overall_Accuracy"]]


In [ ]:
def map_tag_location_accuracy(tag_accuracy_df):
    """
    Map tags to body location and compute overall accuracy per location.
    """

    # Define mapping
    location_map = {
        'At the Ankle': ['7001337', '7001263'],
        'In the Pocket': ['7000480', '7000286'],
        'At the Front': ['7000140', '7000589', '7001084', '7001399'],
        'In the Back': ['7000730', '7000484', '7001235', '7000256']
    }

    # Reverse mapping: Tag_id -> location
    tag_to_location = {}
    for loc, tags in location_map.items():
        for t in tags:
            tag_to_location[str(t)] = loc

    df = tag_accuracy_df.copy()

    # Ensure Tag_id is string for mapping
    df['Tag_id'] = df['Tag_id'].astype(str)
    df['Location'] = df['Tag_id'].map(tag_to_location)

    # Drop rows without location mapping
    df = df.dropna(subset=['Location'])

    # Compute overall accuracy per location (weighted by Total_Points if available)
    models = [col for col in df.columns if col.endswith("_Overall_Accuracy")]

    location_acc = df.groupby('Location').apply(
        lambda x: pd.Series({m: (x[m] * x['Total_Points']).sum() / x['Total_Points'].sum() for m in models})
    ).reset_index()

    return location_acc


In [ ]:
location_accuracy_df = map_tag_location_accuracy(overall_df)
location_accuracy_df

In [ ]:
def plot_rssi_box_by_zone_location(df, location_map, top_n=5):
    
    rssi_cols = [c for c in df.columns if c[0].isdigit()]  
    
    tag_to_location = {str(t): loc for loc, tags in location_map.items() for t in tags}
    df['Tag_id'] = df['tagId'].astype(str)
    df['Location'] = df['Tag_id'].map(tag_to_location)
    df = df.dropna(subset=['Location'])
    
    plot_data = []
    x_labels = []
    
    colors = {'At the Front':'green', 'In the Back':'orange', 'In the Pocket':'red', 'At the Ankle':'blue'}
    
    for zone_id, zone_df in df.groupby('Zone_id'):
        # Compute top RSSI features per zone, excluding -100
        mean_rssi = zone_df[rssi_cols].where(zone_df[rssi_cols] != -100).mean()
        top_features = mean_rssi.sort_values(ascending=False).head(top_n).index.tolist()
        
        for loc in ['At the Front', 'In the Back', 'In the Pocket', 'At the Ankle']:
            loc_df = zone_df[zone_df['Location'] == loc]
            if loc_df.empty:
                plot_data.append([np.nan]*top_n)
            else:
                # Collect only values != -100
                values = loc_df[top_features].applymap(lambda x: x if x != -100 else np.nan)
                # Flatten all non-NaN values
                values_flat = values.values.flatten()
                values_flat = values_flat[~np.isnan(values_flat)]  # remove NaNs
                if len(values_flat) == 0:
                    values_flat = [np.nan] * top_n  # keep placeholder so box is drawn
                plot_data.append(values_flat)

            
            # Only zone ID in x-axis label
            x_labels.append(f"Zone {zone_id}")
    
    # Plot
    plt.figure(figsize=(12, 6))
    box = plt.boxplot(plot_data, patch_artist=True, labels=x_labels)
    
    # Color each box according to location order
    for patch, label, idx in zip(box['boxes'], x_labels, range(len(x_labels))):
        loc = ['At the Front', 'In the Back', 'In the Pocket', 'At the Ankle'][idx % 4]
        patch.set_facecolor(colors.get(loc, 'grey'))

    # Add location legend
    legend_handles = [
    plt.Line2D([0], [0], color=color, lw=8, label=loc)
    for loc, color in colors.items()
    ]

    plt.legend(
        handles=legend_handles,
        title="Tag Location",
        loc="lower center",
        bbox_to_anchor=(0.5, 0.0),
        ncol=4
    )

    plt.ylabel("RSSI")
    plt.title(f"Top {top_n} RSSI Features per Zone by Tag Location")
    plt.xticks(rotation=90, ha='right')
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

In [ ]:
location_map = {
        'At the Ankle': ['7001337', '7001263'],
        'In the Pocket': ['7000480', '7000286'],
        'At the Front': ['7000140', '7000589', '7001084', '7001399'],
        'In the Back': ['7000730', '7000484', '7001235', '7000256']
    }

In [ ]:
plot_rssi_box_by_zone_location(data_set_df, location_map)

## C. Using 5 dp
Using 5 dp and apply the Centroid for location


In [ ]:
from ast import literal_eval

def safe_eval(x):
    if isinstance(x, str):
        return literal_eval(x)
    return x

def compute_centroid(points):
    xs, ys = zip(*points)
    return [np.mean(xs), np.mean(ys)]

In [ ]:
def compute_centroids_by_window(result, window_sizes=range(1, 11)):
    result = result.copy()

    # Safely evaluate string lists
    result['Predicted_MLE'] = result['Predicted_MLE'].apply(safe_eval)
    result['Ground_Truth'] = result['Ground_Truth'].apply(safe_eval)

    # Handle naming differences
    if 'Predicted_Optimisation' in result.columns:
        optimisation_col = 'Predicted_Optimisation'
    elif 'Predicted_Opt' in result.columns:
        optimisation_col = 'Predicted_Opt'
    else:
        raise KeyError("Neither 'Predicted_Optimisation' nor 'Predicted_Opt' found in DataFrame")

    result[optimisation_col] = result[optimisation_col].apply(safe_eval)

    # Optional fused predictions
    has_fused = 'Predicted_NLOS' in result.columns
    if has_fused:
        result['Predicted_NLOS'] = result['Predicted_NLOS'].apply(safe_eval)

    all_results = []

    group_cols = ['Zone_id', 'Room_name', 'Tag_id']

    for window_size in window_sizes:
        for group_keys, group in result.groupby(group_cols):
            group = group.sort_values('timestamp').reset_index(drop=True)
            n = len(group)

            for i in range(n):

                # ----------- CORRECTED WINDOW LOGIC ---------------
                if i < window_size:
                    # BEGINNING: grow window
                    start = 0
                    end = i + 1
                else:
                    # SLIDING WINDOW: always full windows
                    start = i - window_size + 1
                    end = i + 1
                # --------------------------------------------------

                window = group.iloc[start:end]

                # Extract points
                mle_points = list(window['Predicted_MLE'])
                optimisation_points = list(window[optimisation_col])
                ground_truth_points = list(window['Ground_Truth'])

                mle_centroid = compute_centroid(mle_points)
                optimisation_centroid = compute_centroid(optimisation_points)
                ground_truth_centroid = compute_centroid(ground_truth_points)

                result_row = {
                    'Zone_id': group_keys[0],
                    'Room_name': group_keys[1],
                    'tagId': group_keys[2],
                    'Window_Size': window_size,
                    'Predicted_MLE': mle_centroid,
                    'Predicted_Optimisation': optimisation_centroid,
                    'Ground_Truth': ground_truth_centroid
                }

                if has_fused:
                    fused_points = list(window['Predicted_NLOS'])
                    fused_centroid = compute_centroid(fused_points)
                    result_row['Predicted_NLOS'] = fused_centroid

                all_results.append(result_row)

    centroid_df = pd.DataFrame(all_results)
    return centroid_df[centroid_df['Room_name'] != 'Womens Restroom']



In [ ]:
from shapely.geometry import Polygon


In [ ]:
centroid_result = compute_centroids_by_window(result, window_sizes=range(5, 6))
centroid_result.head(1)

In [ ]:
accuracy_df_centroid_5 = plot_predicted_all(centroid_result[centroid_result.Window_Size==5], \
                                            ground_truth_df, map_file)

In [ ]:
# Group by Room_Type and compute weighted (point-based) accuracy
weighted_grouped = (
    accuracy_df_centroid_5
    .groupby("Room_Type")
    .apply(lambda g: pd.Series({
        "MLE_Accuracy": (g["MLE_Inside_Points"].sum() / g["Total_Points"].sum()) * 100,
        "Optimisation_Accuracy": (g["Optimisation_Inside_Points"].sum() / g["Total_Points"].sum()) * 100,
        "NLOS_Accuracy": (g["NLOS_Inside_Points"].sum() / g["Total_Points"].sum()) * 100
    }))
)

# Calculate overall accuracy (also weighted)
overall = pd.DataFrame([{
    "MLE_Accuracy": (accuracy_df_centroid_5["MLE_Inside_Points"].sum() / accuracy_df_centroid_5["Total_Points"].sum()) * 100,
    "Optimisation_Accuracy": (accuracy_df_centroid_5["Optimisation_Inside_Points"].sum() / accuracy_df_centroid_5["Total_Points"].sum()) * 100,
    "NLOS_Accuracy": (accuracy_df_centroid_5["NLOS_Inside_Points"].sum() / accuracy_df_centroid_5["Total_Points"].sum()) * 100
}], index=["Overall"])

# Combine results
summary_df = pd.concat([overall, weighted_grouped]).rename(index={"Open": "Open Space"})
summary_df.to_csv("Result_Duress/temp_result.csv", index= False)
summary_df

In [ ]:
def compute_accuracy_by_window(result, ground_truth_df, map_file_location):

    accuracy_summary = []

    centroid_df = compute_centroids_by_window(result, window_sizes=range(1, 11))
    
    # Add Room_Type to centroid_df by merging
    centroid_df = pd.merge(centroid_df, ground_truth_df[['Zone_id', 'Room_name', 'Room_Type']].drop_duplicates(),
                           on=['Zone_id', 'Room_name'], how='left')

    for w in sorted(centroid_df['Window_Size'].unique()):
        df_w = centroid_df[centroid_df['Window_Size'] == w]

        # Patch: ensure Room_Type is present
        df_w = pd.merge(
            df_w,
            ground_truth_df[['Zone_id', 'Room_name', 'Room_Type']].drop_duplicates(),
            on=['Zone_id', 'Room_name'],
            how='left'
        )

        acc_df = plot_predicted_all(
            result=df_w,
            ground_truth_df=ground_truth_df,
            map_file_location=map_file_location,
#             output_file=f"Result_Duress_Eric_Data_Jun25/temp_plot_ws_{w}.png"
        )

        # Overall accuracy
        mle_overall = acc_df['MLE_Accuracy'].mean()
        opt_overall = acc_df['Optimisation_Accuracy'].mean()
        fused_overall = acc_df['NLOS_Accuracy'].mean()

        grouped = acc_df.groupby("Room_Type")[["MLE_Accuracy", "Optimisation_Accuracy",\
                                              'NLOS_Accuracy']].mean()

        row = {
            "Window_Size": w,
            "MLE_Overall": mle_overall,
            "Optimisation_Overall": opt_overall,
            "NLOS_Overall": fused_overall,
        }

        for room_type in grouped.index:
            row[f"MLE_{room_type}"] = grouped.loc[room_type, "MLE_Accuracy"]
            row[f"Optimisation_{room_type}"] = grouped.loc[room_type, "Optimisation_Accuracy"]
            row[f"NLOS_{room_type}"] = grouped.loc[room_type, "NLOS_Accuracy"]

        accuracy_summary.append(row)

    return pd.DataFrame(accuracy_summary)

In [ ]:
def compute_accuracy_by_window_with_tags(result, ground_truth_df, map_file_location):

    accuracy_summary = []

    centroid_df = compute_centroids_by_window(result, window_sizes=range(1, 11))
    
    # Add Room_Type to centroid_df by merging
    centroid_df = pd.merge(
        centroid_df,
        ground_truth_df[['Zone_id', 'Room_name', 'Room_Type']].drop_duplicates(),
        on=['Zone_id', 'Room_name'],
        how='left'
    )

    for w in sorted(centroid_df['Window_Size'].unique()):
        df_w = centroid_df[centroid_df['Window_Size'] == w]

        # Patch: ensure Room_Type is present
        df_w = pd.merge(
            df_w,
            ground_truth_df[['Zone_id', 'Room_name', 'Room_Type']].drop_duplicates(),
            on=['Zone_id', 'Room_name'],
            how='left'
        )

        # Group by tagId to get per-tag accuracy
        for tag_id, df_tag in df_w.groupby("tagId"):

            acc_df = plot_predicted_all(
                result=df_tag,
                ground_truth_df=ground_truth_df,
                map_file_location=map_file_location,
            )

            # Overall accuracy per tag
            mle_overall = acc_df['MLE_Accuracy'].mean()
            opt_overall = acc_df['Optimisation_Accuracy'].mean()
            NLOS_overall = acc_df['NLOS_Accuracy'].mean()

            # Room-type aggregated accuracy
            grouped = acc_df.groupby("Room_Type")[["MLE_Accuracy", "Optimisation_Accuracy", 'NLOS_Accuracy']].mean()

            row = {
                "Window_Size": w,
                "tagId": tag_id,              # <-- include tagId
                "MLE_Overall": mle_overall,
                "Optimisation_Overall": opt_overall,
                "NLOS_Overall": NLOS_overall,
            }

            for room_type in grouped.index:
                row[f"MLE_{room_type}"] = grouped.loc[room_type, "MLE_Accuracy"]
                row[f"Optimisation_{room_type}"] = grouped.loc[room_type, "Optimisation_Accuracy"]
                row[f"NLOS_{room_type}"] = grouped.loc[room_type, "NLOS_Accuracy"]

            accuracy_summary.append(row)

    return pd.DataFrame(accuracy_summary)


In [ ]:
accuracy_vs_window = compute_accuracy_by_window(result, ground_truth_df, map_file)

accuracy_vs_window

In [ ]:
from shapely.geometry import Polygon

accuracy_vs_window_tag = compute_accuracy_by_window_with_tags(result, ground_truth_df, map_file)

accuracy_vs_window_tag

### Notes
accuracy_by_window will calaute all tags combine so we can have global accuracy

accuracy_vs_window_tag will calaute per tag so we can see indiviudal tag performance

In [ ]:
result.Zone_id.unique()

In [ ]:
df = result[result.Zone_id=='35181'].copy()

# 1. Sort the data
df = df.sort_values(["Tag_id", "timestamp"])


# -------------------------------------------------------
# 2. Expand list columns into x/y numeric columns
# -------------------------------------------------------
def expand_xy(df, col):
    df[[f"{col}_x", f"{col}_y"]] = pd.DataFrame(df[col].tolist(), index=df.index)

expand_xy(df, "Predicted_MLE")
expand_xy(df, "Predicted_Optimisation")
expand_xy(df, "Predicted_NLOS")


# -------------------------------------------------------
# 3. Apply sliding window = 5 to get centroid
# -------------------------------------------------------
window = 5

# MLE centroid
df["MLE_centroid_x"] = df.groupby("Tag_id")["Predicted_MLE_x"].rolling(window).mean().reset_index(level=0, drop=True)
df["MLE_centroid_y"] = df.groupby("Tag_id")["Predicted_MLE_y"].rolling(window).mean().reset_index(level=0, drop=True)

# Optimisation centroid
df["Opt_centroid_x"] = df.groupby("Tag_id")["Predicted_Optimisation_x"].rolling(window).mean().reset_index(level=0, drop=True)
df["Opt_centroid_y"] = df.groupby("Tag_id")["Predicted_Optimisation_y"].rolling(window).mean().reset_index(level=0, drop=True)

# NLOS centroid
df["NLOS_centroid_x"] = df.groupby("Tag_id")["Predicted_NLOS_x"].rolling(window).mean().reset_index(level=0, drop=True)
df["NLOS_centroid_y"] = df.groupby("Tag_id")["Predicted_NLOS_y"].rolling(window).mean().reset_index(level=0, drop=True)


# -------------------------------------------------------
# 4. Combine x + y back into a single list feature
# -------------------------------------------------------
df["Centroid_MLE"]  = df[["MLE_centroid_x", "MLE_centroid_y"]].values.tolist()
df["Centroid_Opt"]  = df[["Opt_centroid_x", "Opt_centroid_y"]].values.tolist()
df["Centroid_NLOS"] = df[["NLOS_centroid_x", "NLOS_centroid_y"]].values.tolist()


# -------------------------------------------------------
# 5. Output with same-format features
# -------------------------------------------------------
final_df = df[[
    "Tag_id", "timestamp",
    "Centroid_MLE",
    "Centroid_Opt",
    "Centroid_NLOS"
]]


In [ ]:
# location_map = {
#         'At the Ankle': ['7001337', '7001263'],
#         'In the Pocket': ['7000480', '7000286'],
#         'At the Front': ['7000140', '7000589', '7001084', '7001399'],
#         'In the Back': ['7000730', '7000484', '7001235', '7000256']
#     }

In [ ]:
location_map = {
#         'At the Ankle': ['7001337', '7001263'],
        'In the Pocket': ['7000480', '7000286'],
#         'At the Front': ['7000140', '7000589', '7001084', '7001399'],
#         'In the Back': ['7000730', '7000484', '7001235', '7000256']
    }

In [ ]:
from matplotlib.patches import Polygon
from matplotlib.patches import Polygon as mplPolygon

def plot_centroids_on_map(
        df,
        map_file,
        tag_id,
        NLOS_cols=["Centroid_NLOS"],
        title="NLOS_ Tag ",
        output="centroid_tracking_eat.png",
        highlight_zones=None,
        ground_truth_df=None,
        location_map=location_map  # dict mapping locations to tag IDs
    ):
    """
    Plot NLOS centroid locations for a tag over time on the map,
    mark start/end points, show movement direction with arrows,
    dynamically zoom to the trajectory, highlight specified zones,
    and show tag location in the title.
    """

    # Filter dataset for the tag
    tag_df = df[df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if tag_df.empty:
        print(f"No data for Tag {tag_id}")
        return

    # Determine tag location from location_map
    location_str = ""
    if location_map:
        for loc, tag_ids in location_map.items():
            if tag_id in tag_ids:
                location_str = loc
                break

    # Convert timestamp to datetime if not already
    tag_df["timestamp"] = pd.to_datetime(tag_df["timestamp"])
    tag_df["timestamp_eat"] = tag_df["timestamp"] - pd.Timedelta(hours=5)

    # Expand centroid list to x, y columns
    def split_xy(column):
        xy = tag_df[column].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
        tag_df[f"{column}_x"] = xy.apply(lambda v: v[0] if isinstance(v, (list, tuple)) else np.nan)
        tag_df[f"{column}_y"] = xy.apply(lambda v: v[1] if isinstance(v, (list, tuple)) else np.nan)

    for col in NLOS_cols:
        split_xy(col)

    # Load map
    image = mpimg.imread(map_file)
    plt.figure(figsize=(14, 7))
    plt.imshow(image, extent=[0, 65, 0, 28], aspect='auto')

    # --- Highlight zones ---
    if highlight_zones and ground_truth_df is not None:
        for zone_id in highlight_zones:
            room_row = ground_truth_df[ground_truth_df['Zone_id'] == zone_id]
            if not room_row.empty:
                room_row = room_row.iloc[0]
                x_coords = [room_row.get(f'x{i+1}') for i in range(8) if pd.notnull(room_row.get(f'x{i+1}'))]
                y_coords = [room_row.get(f'y{i+1}') for i in range(8) if pd.notnull(room_row.get(f'y{i+1}'))]
                if x_coords and y_coords:
                    coords = list(zip(x_coords, y_coords))
                    polygon_patch = mplPolygon(
                        coords,
                        closed=True,
                        fill=True,
                        facecolor='yellow',
                        alpha=0.3,
                        edgecolor='orange',
                        linewidth=2
                    )
                    plt.gca().add_patch(polygon_patch)

    # Collect all valid points to determine zoom
    all_x, all_y = [], []
    for col in NLOS_cols:
        x_col = f"{col}_x"
        y_col = f"{col}_y"
        x_vals = tag_df[x_col].values
        y_vals = tag_df[y_col].values
        valid_mask = (~np.isnan(x_vals)) & (~np.isnan(y_vals))
        all_x.extend(x_vals[valid_mask])
        all_y.extend(y_vals[valid_mask])

    # Compute dynamic zoom with margin
    if len(all_x) > 0 and len(all_y) > 0:
        margin_x = (max(all_x) - min(all_x)) * 0.05 + 1
        margin_y = (max(all_y) - min(all_y)) * 0.05 + 1
        x_min, x_max = min(all_x) - margin_x, max(all_x) + margin_x
        y_min, y_max = min(all_y) - margin_y, max(all_y) + margin_y
    else:
        x_min, x_max = 30, 65
        y_min, y_max = 0, 28

    # Time progression for color
    t_norm = np.linspace(0, 1, len(tag_df))
    cmap = plt.cm.viridis

    # Plot each centroid column
    for col in NLOS_cols:
        x_col = f"{col}_x"
        y_col = f"{col}_y"

        x = tag_df[x_col].values
        y = tag_df[y_col].values
        valid_mask = (~np.isnan(x)) & (~np.isnan(y))
        x = x[valid_mask]
        y = y[valid_mask]
        t_norm_valid = t_norm[valid_mask]

        if len(x) == 0:
            continue

        # Scatter points colored by time
        plt.scatter(x, y, c=t_norm_valid, s=20, cmap=cmap, marker="s", label=f"{col}")

        # Connect points
        plt.plot(x, y, linewidth=1, alpha=0.7, color='grey')

        # Draw arrows for movement direction
        if len(x) > 1:
            dx = np.diff(x)
            dy = np.diff(y)
            plt.quiver(
                x[:-1], y[:-1], dx, dy,
                scale_units='xy', angles='xy', scale=1,
                width=0.003, color='blue', alpha=0.7
            )

        # Start and end markers
        plt.scatter(x[0], y[0], color='red', s=100, marker='*', label=f"Start {col}")
        plt.scatter(x[-1], y[-1], color='red', s=100, marker='X', label=f"End {col}")

    # Start and end time
    start_time = tag_df["timestamp_eat"].iloc[0].strftime("%H:%M:%S")
    end_time   = tag_df["timestamp_eat"].iloc[-1].strftime("%H:%M:%S")
    plt.text(x_min, y_max + 0.5, f"Time: {start_time} → {end_time}", fontsize=12, color="black")

    # Include tag location in title
    plt.title(f"{title}{tag_id} ({location_str})")
    plt.xlabel("X coordinate")
    plt.ylabel("Y coordinate")
    plt.colorbar(label="Time progression (early-->late)")
    plt.legend(loc='lower right', bbox_to_anchor=(0.25,0.1))

    # Apply dynamic zoom to focus on trajectory
    plt.xlim([x_min, x_max])
    plt.ylim([y_min, y_max])

    plt.tight_layout()
    plt.savefig(output, dpi=300)
    plt.show()


In [ ]:
highlight_zones = ["30558",  "30509", "30553", "35182", "30514", "35181", "30516",\
                    "30512",  "30518", "30533",\
                  "30520", "30524", "30525"]

In [ ]:
result.Tag_id.unique()

In [ ]:
from matplotlib.patches import Polygon as mplPolygon

plot_centroids_on_map(
    df=final_df,
    map_file=map_file,
    tag_id="7000286",
    NLOS_cols=["Centroid_NLOS"],
    output="tag_7000480.png",
    highlight_zones=highlight_zones,
    ground_truth_df=ground_truth_df
)

In [ ]:
output_folder = "Plot_Nov25"
os.makedirs(output_folder, exist_ok=True)

# Loop through all unique tags
for tag in final_df['Tag_id'].unique():
    output_file = os.path.join(output_folder, f"NLOS_{tag}.png")
    
    plot_centroids_on_map(
        df=final_df,
        map_file=map_file,
        tag_id=tag,
        NLOS_cols=["Centroid_NLOS"],
        output=output_file,
        highlight_zones=highlight_zones,
        ground_truth_df=ground_truth_df
    )


In [ ]:
import matplotlib.image as mpimg
from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch
from matplotlib.animation import FuncAnimation, FFMpegWriter
import ast

def animate_centroids_video(
        df,
        map_file,
        tag_id,
        NLOS_cols=["Centroid_NLOS"],
        title="NLOS_Tag",
        output="centroid_tracking.mp4",
        highlight_zones=None,
        ground_truth_df=None,
        location_map=None,
        x_min_fixed=None  # optional fixed minimum x-axis
    ):
    tag_df = df[df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if tag_df.empty:
        print(f"No data for Tag {tag_id}")
        return

    # Determine tag location from location_map
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    full_title = f"{title} {tag_id} ({location_str})"

    # Convert timestamps
    tag_df["timestamp"] = pd.to_datetime(tag_df["timestamp"])
    tag_df["timestamp_eat"] = tag_df["timestamp"] - pd.Timedelta(hours=5)

    # Expand centroid columns
    def split_xy(col):
        xy = tag_df[col].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
        tag_df[f"{col}_x"] = xy.apply(lambda v: v[0] if isinstance(v, (list, tuple)) else np.nan)
        tag_df[f"{col}_y"] = xy.apply(lambda v: v[1] if isinstance(v, (list, tuple)) else np.nan)
    for col in NLOS_cols:
        split_xy(col)

    # Load map: support both path and pre-loaded image
    if isinstance(map_file, str):
        image = mpimg.imread(map_file)
    else:
        image = map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')

    # Highlight zones
    if highlight_zones and ground_truth_df is not None:
        for zone_id in highlight_zones:
            row = ground_truth_df[ground_truth_df['Zone_id']==zone_id]
            if not row.empty:
                row = row.iloc[0]
                x_coords = [row[f"x{i+1}"] for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
                y_coords = [row[f"y{i+1}"] for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]
                if x_coords and y_coords:
                    poly = mplPolygon(list(zip(x_coords,y_coords)), closed=True, fill=True,
                                      facecolor='yellow', alpha=0.3, edgecolor='orange', linewidth=2)
                    ax.add_patch(poly)

    # Scatter placeholders
    scatters = []
    for col in NLOS_cols:
        s = ax.scatter([], [], s=30, label=col, color='red')

        scatters.append(s)

    arrows_all = [[] for _ in NLOS_cols]

    # Animation update
    def update(frame):
        for i, col in enumerate(NLOS_cols):
            x = tag_df[f"{col}_x"].values[:frame+1]
            y = tag_df[f"{col}_y"].values[:frame+1]
            mask = (~np.isnan(x)) & (~np.isnan(y))
            x = x[mask]
            y = y[mask]
            scatters[i].set_offsets(np.column_stack((x,y)) if len(x)>0 else np.empty((0,2)))

            # Remove previous arrows
            for arr in arrows_all[i]:
                arr.remove()
            arrows_all[i] = []

            if len(x) > 1:
                for j in range(len(x)-1):
                    arr = FancyArrowPatch((x[j], y[j]), (x[j+1], y[j+1]),
                                          arrowstyle='->', color='red', mutation_scale=10, alpha=0.7)
                    ax.add_patch(arr)
                    arrows_all[i].append(arr)
        return scatters + sum(arrows_all, [])

    ani = FuncAnimation(fig, update, frames=len(tag_df), interval=200, blit=True)

    # Axis limits
    if x_min_fixed is not None:
        ax.set_xlim(left=x_min_fixed)
    ax.set_ylim([0,28])

    ax.set_title(full_title)
    ax.set_xlabel("")
    ax.set_ylabel("")

    # Save video
    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)
    print(f"Video saved as {output}")


In [ ]:
def animate_all_tags(
        df,
        map_file=map_file,
        NLOS_cols=["Centroid_NLOS"],
        title="NLOS_Tag",
        output_folder="videos_Nov25",
        highlight_zones=highlight_zones,
        ground_truth_df=ground_truth_df,
        location_map=location_map,
        x_min_fixed=0  # set x-axis start
    ):
    """
    Animate all tags in the dataframe and save each as an MP4 video
    in the specified output folder, including tag location in the title.
    """
    os.makedirs(output_folder, exist_ok=True)
    tag_ids = df["Tag_id"].unique()

    for tag_id in tag_ids:
        # Determine tag location from location_map
        location_str = ""
        if location_map:
            for loc, tag_list in location_map.items():
                if tag_id in tag_list:
                    location_str = loc
                    break

        video_title = f"{title}"
        output_path = os.path.join(output_folder, f"{title}_{tag_id}_{location_str}.mp4")
        print(f"Processing Tag {tag_id} → {output_path}")

        # Animate the video
        animate_centroids_video(
            df=df,
            map_file=map_file,
            tag_id=tag_id,
            NLOS_cols=NLOS_cols,
            title=video_title,
            output=output_path,
            highlight_zones=highlight_zones,
            ground_truth_df=ground_truth_df,
            location_map=location_map,
            x_min_fixed=x_min_fixed  # pass the fixed x_min
        )


In [ ]:
animate_all_tags(final_df, map_file=map_file1)

## Location AI

In [ ]:
filepath = "data/Duress/2_data_duress_access_Jun23.json"
filename = os.path.basename(filepath)

X1 = process_training(filepath)
df2 = pd.DataFrame(X1)
df2 = df2.fillna(MISSING_VALUE)  

float_cols = df2.select_dtypes(include=['float']).columns
df2[float_cols] = df2[float_cols].astype(np.int8)
df2['timestamp'] = pd.to_datetime(df2['timestamp'], utc=True, errors='coerce')
df2 = df2.sort_values(by='timestamp')

ordered_columns = ['timestamp', 'tagId', 'Zone_id', 'Room_name', "parent_zone_id"]

columns = [col for col in anchor_point_df.Mac.unique().tolist() if col not in ordered_columns]
new_column_order = columns + ordered_columns
df2 = df2.reindex(columns=new_column_order)
df2 = df2.reset_index(drop=True)
df2['Room_name'] = df2['Room_name'].str.split('-').str[-1].str.strip()

# Fix specific zone_id
df2.loc[df2['Zone_id'] == "30598", 'Zone_id'] = "30539"

# Beacon processing
beacon_cols = [col for col in df2.columns if str(col).startswith('0')]
df2 = df2.fillna(MISSING_VALUE)
df2['beacon_count'] = (df2[beacon_cols] != -100).sum(axis=1)
print(df2.shape)

df2 = df2[df2['beacon_count'] >= 5]
print(df2.shape)

In [ ]:
columns_to_drop= anchor_point_df[anchor_point_df.Remove=="remove"].Mac.tolist()
len(columns_to_drop)

In [ ]:
df2= df2.drop(columns= columns_to_drop)

# Find common columns
common_cols = df2.columns.intersection(data_set_df.columns)

# Keep only common columns in df1
df2_aligned = df2[common_cols].copy()
df2_aligned["parent_zone_id"]=df2["parent_zone_id"]

train_data, test_data = train_test_split(df2_aligned, test_size=0.2, random_state=42, \
                                         stratify=df2_aligned["Zone_id"])

X_train = train_data[[col for col in train_data.columns if col.startswith("0")]]
y_train_floor = train_data['parent_zone_id'] 
y_train = train_data['Zone_id']

X_test = df1[[col for col in train_data.columns if col.startswith("0")]] 
y_test_floor = df1['parent_zone_id'] 
y_test = df1['Zone_id'] 
df2_aligned.shape, df2.shape, df1.shape 

In [ ]:
selected_features, clf_rooms, clf_floor = train_variable(X_train, y_train_floor, y_train, save_models = False)
predicted_rooms, predicted_floors = predict_variable(X_test, clf_floor, clf_rooms, selected_features)
score = accuracy_score(y_test, predicted_rooms)
print('Room Accuracy: {:.2f}%'.format(score * 100))

In [ ]:
df1.shape, data_set_df.shape

In [ ]:
result_d = data_set_df[["Room_name", 'tagId', 'Zone_id', "timestamp"]]\
    .merge(ground_truth_df[["Zone_id", "Room_Type"]], on = "Zone_id", how="left")
result_d["Prediction"] = predicted_rooms
result_d["Accuracy"] = np.where(result_d.Zone_id == result_d.Prediction, 100, 0)
result_d.head(1)

In [ ]:
from collections import Counter

def sliding_window_voting(df, window=5):
    """
    Perform sliding window majority vote per Tag_id on 'Prediction'.
    Tie-breaker: latest occurrence in the window.
    
    Returns dataframe with: Tag_id, timestamp, voted_prediction
    """
    
    # Sort by Tag_id and timestamp
    df = df.sort_values(["tagId", "timestamp"]).copy()
    
    results = []

    for tag, group in df.groupby("tagId"):
        preds = group["Prediction"].tolist()
        timestamps = group["timestamp"].tolist()
        
        # Sliding window
        for i in range(len(preds)):
            start_idx = max(0, i - window + 1)
            window_preds = preds[start_idx:i+1]
            
            # Count occurrences
            counts = Counter(window_preds)
            max_count = max(counts.values())
            # Candidates with max votes
            candidates = [k for k, v in counts.items() if v == max_count]
            
            # Tie-breaker: select the latest occurrence in the window
            for val in reversed(window_preds):
                if val in candidates:
                    voted = val
                    break
            
            results.append({
                "tagId": tag,
                "timestamp": timestamps[i],
                "voted_prediction": voted
            })
    
    return pd.DataFrame(results)


In [ ]:
voted_df = sliding_window_voting(result_d, window=5)
# voted_df

In [ ]:
voted_df.head(1)

In [ ]:
def plot_map_with_zones(
    ground_truth_df,
    map_file,
    highlight_zones=None,
    output_file="map_zones.png",
    title="Map with Zones",
    location_str=""
):
    """
    Plot the map with zones highlighted.
    Only shows the map and zone polygons.
    """
    fig, ax = plt.subplots(figsize=(14, 7))
    image = plt.imread(map_file)
    ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')

    if highlight_zones is None:
        highlight_zones = []

    # Draw all zones
    for _, row in ground_truth_df.iterrows():
        zone_id = row['Zone_id']
        x_coords = [row.get(f'x{i+1}') for i in range(8) if pd.notnull(row.get(f'x{i+1}'))]
        y_coords = [row.get(f'y{i+1}') for i in range(8) if pd.notnull(row.get(f'y{i+1}'))]
        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))
            color = 'yellow' if zone_id in highlight_zones else 'none'
            edge_color = 'red' if zone_id in highlight_zones else 'black'
            poly = mplPolygon(coords, closed=True, fill=(color!='none'),
                              facecolor=color, edgecolor=edge_color,
                              alpha=0.3 if color!='none' else 0.8, linewidth=2)
            ax.add_patch(poly)

    # Set title
    if location_str:
        ax.set_title(f"{title} ({location_str})")
    else:
        ax.set_title(title)

#     ax.set_xlabel("X coordinate")
#     ax.set_ylabel("Y coordinate")
    ax.set_xlim([0, 65])
    ax.set_ylim([0, 28])
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()


In [ ]:
plot_map_with_zones(
    ground_truth_df,
    map_file,
    highlight_zones=highlight_zones,
    output_file="map_zones.png",
    title=None,
    location_str=""
)

In [ ]:
def plot_voted_trajectory_with_time(
    voted_df,
    ground_truth_df,
    map_file,
    location_map=location_map,      # dict of locations
    output_file="tag_trajectory_time.png",
    highlight_zones=None,
    title=None
):
    """
    Plot tag trajectories with:
    - Dwell counts at zone centroids
    - Transition counts along arrows
    - Start (*) and end (X) markers
    - Arrow colors show time progression (early → late)
    - Highlight specified zones
    - Include tag location in the title if location_map provided
    """

    fig, ax = plt.subplots(figsize=(14, 7))
    image = plt.imread(map_file)
    ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')

    if highlight_zones is None:
        highlight_zones = []

    # Draw all zones with optional highlight
    for _, row in ground_truth_df.iterrows():
        zone_id = row['Zone_id']
        x_coords = [row.get(f'x{i+1}') for i in range(8) if pd.notnull(row.get(f'x{i+1}'))]
        y_coords = [row.get(f'y{i+1}') for i in range(8) if pd.notnull(row.get(f'y{i+1}'))]
        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))
            if zone_id in highlight_zones:
                polygon_patch = mplPolygon(coords, closed=True, fill=True,
                                           facecolor='yellow', edgecolor='red',
                                           alpha=0.3, linewidth=2)
            else:
                polygon_patch = mplPolygon(coords, closed=True, fill=False,
                                           edgecolor='black', linewidth=1, alpha=0.8)
            ax.add_patch(polygon_patch)

    cmap = plt.cm.viridis

    for tag, group in voted_df.groupby('tagId'):
        group = group.sort_values('timestamp').reset_index(drop=True)
        zones_visited = list(group['voted_prediction'])

        # Determine location from location_map
        location_str = ""
        if location_map:
            for loc, tag_ids in location_map.items():
                if tag in tag_ids:
                    location_str = loc
                    break

        # Extract centroids for zones
        zone_centroids = {}
        for zone in set(zones_visited):
            room_row = ground_truth_df[ground_truth_df['Zone_id'] == zone].iloc[0]
            x_coords = [room_row.get(f'x{i+1}') for i in range(8) if pd.notnull(room_row.get(f'x{i+1}'))]
            y_coords = [room_row.get(f'y{i+1}') for i in range(8) if pd.notnull(room_row.get(f'y{i+1}'))]
            if x_coords and y_coords:
                polygon = Polygon(list(zip(x_coords, y_coords)))
                if not polygon.is_valid:
                    polygon = polygon.buffer(0)
                centroid = polygon.centroid
                zone_centroids[zone] = (centroid.x, centroid.y)

        # Dwell and transition counts
        from collections import defaultdict, Counter
        dwell_counts = defaultdict(int)
        i = 0
        while i < len(zones_visited):
            current_zone = zones_visited[i]
            count = 1
            while i + count < len(zones_visited) and zones_visited[i + count] == current_zone:
                count += 1
            dwell_counts[current_zone] += count
            i += count

        transition_counts = Counter()
        for i in range(len(zones_visited)-1):
            pair = (zones_visited[i], zones_visited[i+1])
            transition_counts[pair] += 1

        # Time normalization for arrows
        timestamps = pd.to_datetime(group['timestamp'])
        t_norm = (timestamps - timestamps.min()) / (timestamps.max() - timestamps.min())

        # Draw arrows with time color
        for i in range(len(zones_visited)-1):
            z1, z2 = zones_visited[i], zones_visited[i+1]
            x1, y1 = zone_centroids[z1]
            x2, y2 = zone_centroids[z2]
            arrow_color = cmap(t_norm.iloc[i])
            arrow = FancyArrowPatch((x1, y1), (x2, y2),
                                    arrowstyle='->', color=arrow_color,
                                    mutation_scale=15, linewidth=2.5, alpha=0.8)
            ax.add_patch(arrow)
            # Annotate transition count
            count = transition_counts[(z1, z2)]
            dx, dy = x2 - x1, y2 - y1
            xm, ym = x1 + 0.4*dx - 0.3*dy/np.hypot(dx, dy), y1 + 0.4*dy + 0.3*dx/np.hypot(dx, dy)
            ax.text(xm, ym, str(count), color='purple', fontsize=12, fontweight='bold',
                    ha='center', va='center')

        # Dwell counts at centroids
        for zone, count in dwell_counts.items():
            x, y = zone_centroids[zone]
            ax.text(x, y + 0.5, str(count), color='blue', fontsize=14,
                    ha='center', va='center')

        # Start/end markers
        start_zone = zones_visited[0]
        end_zone = zones_visited[-1]
        ax.scatter(*zone_centroids[start_zone], color='red', s=100, marker='*', label=f"Start Tag {tag}")
        ax.scatter(*zone_centroids[end_zone], color='red', s=100, marker='X', label=f"End Tag {tag}")

        # Title includes tag id and location
        ax.set_title(f"LocationAI_ Tag {tag} ({location_str})")

    ax.set_xlabel("X coordinate")
    ax.set_ylabel("Y coordinate")
    ax.set_xlim([0, 65])
    ax.set_ylim([0, 28])
    plt.colorbar(plt.cm.ScalarMappable(cmap=cmap), ax=ax, label="Time progression (early → late)")
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()

from shapely.geometry import Polygon
from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch

In [ ]:
# highlight_zones = ["30543", "30512", "30513", "30515", "30540", "30518", "30519", \
#                   "30520", "30525", "30526", "30538", "30539"]  # zones to highlight

plot_voted_trajectory_with_time(
    voted_df=voted_df[voted_df.tagId=="7000480"],
    ground_truth_df=ground_truth_df,
    map_file=map_file,
#     output_file="tag_voted_trajectory_highlight.png",
    highlight_zones=highlight_zones
)


In [ ]:
output_folder = "Plot_LocationAI_Nov25"
os.makedirs(output_folder, exist_ok=True)  

for tag_id, tag_df in voted_df.groupby("tagId"):
    output_file = os.path.join(output_folder, f"LocationAI_tag_{tag_id}.png")
    
    plot_voted_trajectory_with_time(
        voted_df=tag_df,
        ground_truth_df=ground_truth_df,
        map_file=map_file,
        output_file=output_file,
        highlight_zones=highlight_zones,
        title=f"LocationAI_ Tag {tag_id}"
    )



In [ ]:
import matplotlib as mpl

# Set embed limit to unlimited
mpl.rcParams['animation.embed_limit'] = 2**30  # ~1 GB (effectively unlimited)

In [ ]:
from matplotlib.patches import Polygon as mplPolygon
from matplotlib.animation import FuncAnimation
from PIL import Image

def animate_voted_trajectory(tag_df, ground_truth_df, map_file,
                             highlight_zones=None,
                             output_file="trajectory.mp4",
                             fps=5,
                             location_str=""):
    """
    Animate tag trajectory with dwell/transition counts and optional zone highlights.
    
    Parameters:
    - tag_df: DataFrame containing tag trajectory with 'voted_prediction' and 'timestamp'.
    - ground_truth_df: DataFrame containing zone coordinates and 'Zone_id'.
    - map_file: either file path to image OR a pre-loaded NumPy array (image).
    - highlight_zones: list of Zone_ids to highlight.
    - output_file: file path to save the animation (mp4).
    - fps: frames per second for animation.
    - location_str: optional string to include in the title.
    
    Features:
    - Zones drawn with optional highlights
    - Trajectory plotted as line
    - Start marker: '*', End marker: 'X'
    """

    fig, ax = plt.subplots(figsize=(14, 7))

    # Load image if map_file is a path, otherwise assume it's an array
    if isinstance(map_file, str):
        image = plt.imread(map_file)
    else:
        image = map_file

    ax.imshow(image, extent=[0, 65, 0, 28], aspect='auto')

    if highlight_zones is None:
        highlight_zones = []

    # Draw zones and compute centroids
    zone_centroids = {}
    for _, row in ground_truth_df.iterrows():
        zone_id = row['Zone_id']
        x_coords = [row.get(f'x{i+1}') for i in range(8) if pd.notnull(row.get(f'x{i+1}'))]
        y_coords = [row.get(f'y{i+1}') for i in range(8) if pd.notnull(row.get(f'y{i+1}'))]
        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))
            color = 'yellow' if zone_id in highlight_zones else 'none'
            edge_color = 'red' if zone_id in highlight_zones else 'black'
            poly = mplPolygon(coords, closed=True, fill=(color != 'none'),
                              facecolor=color, edgecolor=edge_color,
                              alpha=0.3 if color != 'none' else 0.8)
            ax.add_patch(poly)
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

    # Prepare trajectory line and markers
    zones_visited = list(tag_df.sort_values('timestamp')['voted_prediction'])
    x = [zone_centroids[z][0] for z in zones_visited]
    y = [zone_centroids[z][1] for z in zones_visited]

    traj_line, = ax.plot([], [], lw=2, color='red')
    start_marker = ax.scatter([], [], s=100, marker='*', color='red')
    end_marker = ax.scatter([], [], s=100, marker='X', color='red')

    def init():
        traj_line.set_data([], [])
        start_marker.set_offsets([[0, 0]])
        end_marker.set_offsets([[0, 0]])
        return traj_line, start_marker, end_marker

    def update(frame):
        traj_line.set_data(x[:frame+1], y[:frame+1])
        start_marker.set_offsets([x[0], y[0]])
        end_marker.set_offsets([x[frame], y[frame]])
        return traj_line, start_marker, end_marker

    anim = FuncAnimation(fig, update, frames=len(x), init_func=init,
                         blit=True, repeat=False)

    ax.set_xlim([0, 65])
    ax.set_ylim([0, 28])

    # Set title with tag ID and location
    tag_id = tag_df['tagId'].iloc[0]
    if location_str:
        ax.set_title(f"LocationAI_ Tag {tag_id} ({location_str})")
    else:
        ax.set_title(f"Animated trajectory for Tag {tag_id}")

    if output_file:
        anim.save(output_file, fps=fps, dpi=200)

    plt.show()
    return anim


In [ ]:
map_file1 = mpimg.imread("map_file1.png")

# Display it
plt.figure(figsize=(8,6))
plt.imshow(map_file1)
plt.axis('off')  # hide axes
plt.show()

In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Polygon as mplPolygon


anim = animate_voted_trajectory(
    voted_df[voted_df.tagId=="7000480"],
    ground_truth_df,
    map_file1
)

HTML(anim.to_jshtml())  

In [ ]:
def animate_multiple_tags(voted_df, ground_truth_df, map_file, location_map,
                                        output_folder="tag_videos", highlight_zones=None, fps=5):
    os.makedirs(output_folder, exist_ok=True)
    tag_ids = voted_df['tagId'].unique()
    for tag in tag_ids:
        print(f"Processing Tag {tag}...")
        tag_df = voted_df[voted_df['tagId'] == tag].sort_values('timestamp')
        if tag_df.empty:
            print(f"No data for Tag {tag}, skipping.")
            continue

        # Determine location from location_map
        location_str = ""
        for loc, tags in location_map.items():
            if tag in tags:
                location_str = loc
                break

        output_file = os.path.join(output_folder, f"LocationAI_Tag_{tag}_{location_str}.mp4")

        # Pass location_str to the animation
        animate_voted_trajectory(
            tag_df,
            ground_truth_df,
            map_file,
            highlight_zones=highlight_zones,
            output_file=output_file,
            fps=fps,
            location_str=location_str
        )

        print(f"Saved animation → {output_file}")


In [ ]:
animate_multiple_tags(
    voted_df=voted_df,
    ground_truth_df=ground_truth_df,
    map_file=map_file1,
    location_map=location_map,
    output_folder="videos_Nov25",
    highlight_zones=highlight_zones,
    fps=5
)


## Both Videos Vertical

In [ ]:
import os
import subprocess

def merge_vertical(video_top, video_bottom, output_video):
    import os, subprocess

    out_dir = os.path.dirname(output_video)
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)

    # Use ffmpeg scale filter to match widths
    cmd = [
        "ffmpeg", "-y",
        "-i", video_top,
        "-i", video_bottom,
        "-filter_complex",
        "[1:v]scale=2800:-1[v1];[0:v][v1]vstack=inputs=2",
        "-c:v", "libx264", "-crf", "18", "-preset", "medium",
        output_video
    ]
    subprocess.run(cmd, check=True)
    print(f"Generated vertical comparison → {output_video}")



In [ ]:
def auto_merge_all_pairs_vertical(videos_folder="videos_Nov25"):
    files = os.listdir(videos_folder)

    # Regex to extract components
    pattern = r"(LocationAI|NLOS)_Tag_(\d+)_(.*)\.mp4"

    # Store discovered files
    locationAI_files = {}
    nlos_files = {}

    for f in files:
        match = re.match(pattern, f)
        if match:
            prefix, tag, location = match.groups()
            key = (tag, location)

            if prefix == "LocationAI":
                locationAI_files[key] = os.path.join(videos_folder, f)
            else:
                nlos_files[key] = os.path.join(videos_folder, f)

    # Match pairs and merge vertically
    for key in locationAI_files:
        if key in nlos_files:
            tag, location = key
            top = locationAI_files[key]
            bottom = nlos_files[key]

            output_name = f"Duress_{tag}_{location}.mp4"
            output_path = os.path.join(videos_folder, output_name)

            print(f"\n🔍 Pair found for Tag {tag} ({location})")
            print(f"   TOP   : {top}")
            print(f"   BOTTOM: {bottom}")
            print("   → generating vertical comparison...")

            merge_vertical(top, bottom, output_path)
        else:
            print(f"⚠️ Missing NLOS file for Tag {key[0]} ({key[1]})")

    print("\n✅ All available pairs processed.\n")


In [ ]:
auto_merge_all_pairs_vertical(videos_folder="videos_Nov25")

In [ ]:
def animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=None,
        highlight_zones=None,
        output="combined_video.mp4",
        x_min_fixed=None
    ):

    import matplotlib.image as mpimg
    from matplotlib.animation import FuncAnimation, FFMpegWriter
    from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch
    import pandas as pd

    # Extract NLOS data
    df_nlos = nlos_df[nlos_df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if df_nlos.empty:
        print(f"No NLOS data for Tag {tag_id}")
        return

    # Extract LocationAI data
    df_loc = voted_df[voted_df["tagId"] == tag_id].sort_values("timestamp").copy()
    if df_loc.empty:
        print(f"No LocationAI data for Tag {tag_id}")
        return

    # Determine location name
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    # Expand NLOS centroid (assume one column: "Centroid_NLOS")
    xy = df_nlos["Centroid_NLOS"].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
    df_nlos["x"] = xy.apply(lambda v: v[0] if isinstance(v, (list,tuple)) else np.nan)
    df_nlos["y"] = xy.apply(lambda v: v[1] if isinstance(v, (list,tuple)) else np.nan)

    # Load map
    if isinstance(map_file, str):
        image = mpimg.imread(map_file)
    else:
        image = map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')

    # Draw zones
    zone_centroids = {}
    for _, row in ground_truth_df.iterrows():
        zone_id = row['Zone_id']
        x_coords = [row[f"x{i+1}"] for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
        y_coords = [row[f"y{i+1}"] for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]
        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))
            color = "yellow" if highlight_zones and zone_id in highlight_zones else "none"
            edge_color = "orange" if highlight_zones and zone_id in highlight_zones else "black"
            poly = mplPolygon(coords, closed=True, fill=(color!="none"),
                              facecolor=color, edgecolor=edge_color, alpha=0.3)
            ax.add_patch(poly)
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

    # Prepare NLOS scatter + arrows
    nlos_scatter = ax.scatter([], [], s=30, color="red")
    nlos_arrows = []

    # Prepare LocationAI stepwise path
    zone_list = list(df_loc["voted_prediction"])
    loc_x = [zone_centroids[z][0] for z in zone_list]
    loc_y = [zone_centroids[z][1] for z in zone_list]

    loc_line, = ax.plot([], [], color="red", linewidth=2)
    loc_start = ax.scatter([], [], s=100, marker="*", color="blue")
    loc_end   = ax.scatter([], [], s=100, marker="X", color="blue")

    # Animation update
    max_frames = max(len(df_nlos), len(loc_x))

    def update(frame):
        # --- NLOS UPDATE ---
        x_nlos = df_nlos["x"].values[:frame+1]
        y_nlos = df_nlos["y"].values[:frame+1]

        mask = (~np.isnan(x_nlos)) & (~np.isnan(y_nlos))
        x_nlos = x_nlos[mask]
        y_nlos = y_nlos[mask]

        nlos_scatter.set_offsets(np.column_stack((x_nlos, y_nlos)) if len(x_nlos)>0 else np.empty((0,2)))

        # remove old arrows
        for arr in nlos_arrows:
            arr.remove()
        nlos_arrows.clear()

        # draw arrows
        if len(x_nlos) > 1:
            for i in range(len(x_nlos)-1):
                arr = FancyArrowPatch((x_nlos[i], y_nlos[i]),
                                      (x_nlos[i+1], y_nlos[i+1]),
                                      arrowstyle="->", color="red",
                                      mutation_scale=10, alpha=0.7)
                ax.add_patch(arr)
                nlos_arrows.append(arr)

        # --- LocationAI UPDATE (stepwise movement)
        if frame < len(loc_x):
            loc_line.set_data(loc_x[:frame+1], loc_y[:frame+1])
            loc_start.set_offsets([loc_x[0], loc_y[0]])
            loc_end.set_offsets([loc_x[frame], loc_y[frame]])

        return [nlos_scatter, loc_line, loc_start, loc_end] + nlos_arrows

    ani = FuncAnimation(fig, update, frames=max_frames,
                        interval=200, blit=True)

    # Axis setup
    if x_min_fixed is not None:
        ax.set_xlim(left=x_min_fixed)
    ax.set_ylim([0,28])

    ax.set_title(f"LocationAI (Blue) + NLOS (Red)— Tag {tag_id} ({location_str})")

    # Save
    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)

    print(f"Saved combined video → {output}")


In [ ]:
final_df.Tag_id.unique()

In [ ]:
def animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=None,
        highlight_zones=None,
        output_folder="outputs",
        x_min_fixed=None
    ):

    import matplotlib.image as mpimg
    from matplotlib.animation import FuncAnimation, FFMpegWriter
    from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch
    import pandas as pd
    import os

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Final output path (tag_id.mp4)
    output = f"{output_folder}/{tag_id}.mp4"

    # Extract NLOS data
    df_nlos = nlos_df[nlos_df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if df_nlos.empty:
        print(f"No NLOS data for Tag {tag_id}")
        return

    # Extract LocationAI data
    df_loc = voted_df[voted_df["tagId"] == tag_id].sort_values("timestamp").copy()
    if df_loc.empty:
        print(f"No LocationAI data for Tag {tag_id}")
        return

    # Determine location name
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    # Expand NLOS centroid
    xy = df_nlos["Centroid_NLOS"].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
    df_nlos["x"] = xy.apply(lambda v: v[0] if isinstance(v, (list,tuple)) else np.nan)
    df_nlos["y"] = xy.apply(lambda v: v[1] if isinstance(v, (list,tuple)) else np.nan)

    # Load map
    image = mpimg.imread(map_file) if isinstance(map_file, str) else map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')

    # Draw zones
    zone_centroids = {}
    for _, row in ground_truth_df.iterrows():
        zone_id = row['Zone_id']
        x_coords = [row[f"x{i+1}"] for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
        y_coords = [row[f"y{i+1}"] for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]

        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))
            color = "yellow" if highlight_zones and zone_id in highlight_zones else "none"
            edge_color = "orange" if highlight_zones and zone_id in highlight_zones else "black"

            poly = mplPolygon(coords, closed=True, fill=(color!="none"),
                              facecolor=color, edgecolor=edge_color, alpha=0.3)
            ax.add_patch(poly)

            # Compute zone centroid
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

    # -------- NLOS Scatter (Red) --------
    nlos_scatter = ax.scatter([], [], s=20, color="red")
    nlos_arrows = []

    # -------- LocationAI Path (Blue) --------
    zone_list = list(df_loc["voted_prediction"])
    loc_x = [zone_centroids[z][0] for z in zone_list]
    loc_y = [zone_centroids[z][1] for z in zone_list]

    loc_line, = ax.plot([], [], color="blue", linewidth=2)     # BLUE line
    loc_start = ax.scatter([], [], s=100, marker="*", color="blue")  # BLUE start
    loc_end   = ax.scatter([], [], s=100, marker="X", color="blue")  # BLUE end

    # Animation update
    max_frames = max(len(df_nlos), len(loc_x))

    def update(frame):
        # --- NLOS UPDATE ---
        x_nlos = df_nlos["x"].values[:frame+1]
        y_nlos = df_nlos["y"].values[:frame+1]

        mask = (~np.isnan(x_nlos)) & (~np.isnan(y_nlos))
        x_nlos = x_nlos[mask]
        y_nlos = y_nlos[mask]

        nlos_scatter.set_offsets(np.column_stack((x_nlos, y_nlos)) if len(x_nlos)>0 else np.empty((0,2)))

        # Remove previous arrows
        for arr in nlos_arrows:
            arr.remove()
        nlos_arrows.clear()

        # Draw new arrows
        if len(x_nlos) > 1:
            for i in range(len(x_nlos)-1):
                arr = FancyArrowPatch((x_nlos[i], y_nlos[i]),
                                      (x_nlos[i+1], y_nlos[i+1]),
                                      arrowstyle="->", color="red",
                                      mutation_scale=10, alpha=0.7)
                ax.add_patch(arr)
                nlos_arrows.append(arr)

        # --- LocationAI UPDATE (Blue) ---
        if frame < len(loc_x):
            loc_line.set_data(loc_x[:frame+1], loc_y[:frame+1])
            loc_start.set_offsets([loc_x[0], loc_y[0]])
            loc_end.set_offsets([loc_x[frame], loc_y[frame]])

        return [nlos_scatter, loc_line, loc_start, loc_end] + nlos_arrows

    ani = FuncAnimation(fig, update, frames=max_frames, interval=200, blit=True)

    # Axis setup
    if x_min_fixed is not None:
        ax.set_xlim(left=x_min_fixed)
    ax.set_ylim([0,28])

    ax.set_title(f"LocationAI (Blue) + NLOS (Red) — Tag {tag_id} ({location_str})")

    # Save the animation
    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)

    print(f"Saved combined video → {output}")


In [ ]:
animate_combined_results(
        final_df,
        voted_df,
        ground_truth_df,
        map_file1,
        tag_id='7000140',
        location_map=location_map,
        highlight_zones=highlight_zones,
        output_folder="videos",
        x_min_fixed=None
    )

In [ ]:
all_tags = final_df["Tag_id"].unique()

for tag in all_tags:
    print(f"Processing Tag {tag}...")
    animate_combined_results(
        final_df,
        voted_df,
        ground_truth_df,
        map_file1,
        tag_id=tag,
        location_map=location_map,
        highlight_zones=highlight_zones,
        output_folder="videos",
        x_min_fixed=None
    )


## USE THIS to combine both Videso into one

In [ ]:
def animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=None,
        highlight_zones=None,
        output_folder="outputs",
        x_offset=0,
        y_offset=0
    ):

    import matplotlib.image as mpimg
    from matplotlib.animation import FuncAnimation, FFMpegWriter
    from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Final output path (tag_id.mp4)
    output = os.path.join(output_folder, f"{tag_id}.mp4")

    # Extract NLOS data
    df_nlos = nlos_df[nlos_df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if df_nlos.empty:
        print(f"No NLOS data for Tag {tag_id}")
        return

    # Extract LocationAI data
    df_loc = voted_df[voted_df["tagId"] == tag_id].sort_values("timestamp").copy()
    if df_loc.empty:
        print(f"No LocationAI data for Tag {tag_id}")
        return

    # Determine location name
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    # Expand NLOS centroid and apply offsets
    xy = df_nlos["Centroid_NLOS"].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
    df_nlos["x"] = xy.apply(lambda v: v[0] + x_offset if isinstance(v, (list,tuple)) else np.nan)
    df_nlos["y"] = xy.apply(lambda v: v[1] + y_offset if isinstance(v, (list,tuple)) else np.nan)

    # Load map
    image = mpimg.imread(map_file) if isinstance(map_file, str) else map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')

    # Draw zones and compute centroids
    zone_centroids = {}
    for _, row in ground_truth_df.iterrows():
        zone_id = row['Zone_id']
        x_coords = [row[f"x{i+1}"] + x_offset for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
        y_coords = [row[f"y{i+1}"] + y_offset for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]

        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))
            color = "yellow" if highlight_zones and zone_id in highlight_zones else "none"
            edge_color = "orange" if highlight_zones and zone_id in highlight_zones else "black"

            poly = mplPolygon(coords, closed=True, fill=(color!="none"),
                              facecolor=color, edgecolor=edge_color, alpha=0.3)
            ax.add_patch(poly)

            # Compute zone centroid
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

    # -------- NLOS Scatter (Red) --------
    nlos_scatter = ax.scatter([], [], s=20, color="red")
    nlos_arrows = []

    # -------- LocationAI Path (Blue) --------
    zone_list = list(df_loc["voted_prediction"])
    loc_x = [zone_centroids[z][0] for z in zone_list]
    loc_y = [zone_centroids[z][1] for z in zone_list]

    loc_line, = ax.plot([], [], color="blue", linewidth=2)          # BLUE line
    loc_start = ax.scatter([], [], s=100, marker="*", color="blue") # BLUE start
    loc_end   = ax.scatter([], [], s=100, marker="X", color="blue") # BLUE end

    # Animation update
    max_frames = max(len(df_nlos), len(loc_x))

    def update(frame):
        # --- NLOS UPDATE ---
        x_nlos = df_nlos["x"].values[:frame+1]
        y_nlos = df_nlos["y"].values[:frame+1]

        mask = (~np.isnan(x_nlos)) & (~np.isnan(y_nlos))
        x_nlos = x_nlos[mask]
        y_nlos = y_nlos[mask]

        nlos_scatter.set_offsets(np.column_stack((x_nlos, y_nlos)) if len(x_nlos)>0 else np.empty((0,2)))

        # Remove previous arrows
        for arr in nlos_arrows:
            arr.remove()
        nlos_arrows.clear()

        # Draw new arrows
        if len(x_nlos) > 1:
            for i in range(len(x_nlos)-1):
                arr = FancyArrowPatch((x_nlos[i], y_nlos[i]),
                                      (x_nlos[i+1], y_nlos[i+1]),
                                      arrowstyle="->", color="red",
                                      mutation_scale=10, alpha=0.7)
                ax.add_patch(arr)
                nlos_arrows.append(arr)

        # --- LocationAI UPDATE (Blue) ---
        if frame < len(loc_x):
            loc_line.set_data(loc_x[:frame+1], loc_y[:frame+1])
            loc_start.set_offsets([loc_x[0], loc_y[0]])
            loc_end.set_offsets([loc_x[frame], loc_y[frame]])

        return [nlos_scatter, loc_line, loc_start, loc_end] + nlos_arrows

    ani = FuncAnimation(fig, update, frames=max_frames, interval=200, blit=True)

    # Axis setup
    ax.set_xlim([0,65])
    ax.set_ylim([0,28])

    ax.set_title(f"LocationAI (Blue) + NLOS (Red) — Tag {tag_id} ({location_str})")

    # Save the animation
    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)

    print(f"Saved combined video → {output}")

In [ ]:
map_file1 = mpimg.imread("map_file2.png")

# Display it
plt.figure(figsize=(8,6))
plt.imshow(map_file1)
plt.axis('off')  # hide axes
plt.show()

### Add strat and end

In [ ]:
def animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=None,
        highlight_zones=None,
        output_folder="outputs",
        x_offset=0,
        y_offset=0
    ):

    import matplotlib.image as mpimg
    from matplotlib.animation import FuncAnimation, FFMpegWriter
    from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Final output path (tag_id.mp4)
    output = os.path.join(output_folder, f"{tag_id}.mp4")

    # Extract NLOS data
    df_nlos = nlos_df[nlos_df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if df_nlos.empty:
        print(f"No NLOS data for Tag {tag_id}")
        return

    # Extract LocationAI data
    df_loc = voted_df[voted_df["tagId"] == tag_id].sort_values("timestamp").copy()
    if df_loc.empty:
        print(f"No LocationAI data for Tag {tag_id}")
        return

    # Determine location name
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    # Expand NLOS centroid and apply offsets
    xy = df_nlos["Centroid_NLOS"].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
    df_nlos["x"] = xy.apply(lambda v: v[0] + x_offset if isinstance(v, (list,tuple)) else np.nan)
    df_nlos["y"] = xy.apply(lambda v: v[1] + y_offset if isinstance(v, (list,tuple)) else np.nan)

    # Load map
    image = mpimg.imread(map_file) if isinstance(map_file, str) else map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')
    
    
 #################       # ----- Start & End Point Overlay -----
    # Example: replace with your real coordinates
    start_x, start_y = 6, 16   # Start point
    end_x, end_y     = 12, 17  # End point

    start_marker = ax.scatter(start_x, start_y, 
                              s=200, color="green", marker="o", edgecolor="black", linewidth=1.5)
    ax.text(start_x, start_y + 0.5, "Start", color="green", fontsize=12, weight="bold")

    end_marker = ax.scatter(end_x, end_y, 
                            s=200, color="purple", marker="s", edgecolor="black", linewidth=1.5)
    ax.text(end_x, end_y + 0.5, "End", color="purple", fontsize=12, weight="bold")

################

    # Draw zones and compute centroids
    zone_centroids = {}
    for _, row in ground_truth_df.iterrows():
        zone_id = row['Zone_id']
        x_coords = [row[f"x{i+1}"] + x_offset for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
        y_coords = [row[f"y{i+1}"] + y_offset for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]

        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))
            color = "yellow" if highlight_zones and zone_id in highlight_zones else "none"
            edge_color = "orange" if highlight_zones and zone_id in highlight_zones else "black"

            poly = mplPolygon(coords, closed=True, fill=(color!="none"),
                              facecolor=color, edgecolor=edge_color, alpha=0.3)
            ax.add_patch(poly)

            # Compute zone centroid
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

    # -------- NLOS Scatter (Red) --------
    nlos_scatter = ax.scatter([], [], s=20, color="red")
    nlos_arrows = []

    # -------- LocationAI Path (Blue) --------
    zone_list = list(df_loc["voted_prediction"])
    loc_x = [zone_centroids[z][0] for z in zone_list]
    loc_y = [zone_centroids[z][1] for z in zone_list]

    loc_line, = ax.plot([], [], color="blue", linewidth=2)          # BLUE line
    loc_start = ax.scatter([], [], s=100, marker="*", color="blue") # BLUE start
    loc_end   = ax.scatter([], [], s=100, marker="X", color="blue") # BLUE end

    # Animation update
    max_frames = max(len(df_nlos), len(loc_x))

    def update(frame):
        # --- NLOS UPDATE ---
        x_nlos = df_nlos["x"].values[:frame+1]
        y_nlos = df_nlos["y"].values[:frame+1]

        mask = (~np.isnan(x_nlos)) & (~np.isnan(y_nlos))
        x_nlos = x_nlos[mask]
        y_nlos = y_nlos[mask]

        nlos_scatter.set_offsets(np.column_stack((x_nlos, y_nlos)) if len(x_nlos)>0 else np.empty((0,2)))

        # Remove previous arrows
        for arr in nlos_arrows:
            arr.remove()
        nlos_arrows.clear()

        # Draw new arrows
        if len(x_nlos) > 1:
            for i in range(len(x_nlos)-1):
                arr = FancyArrowPatch((x_nlos[i], y_nlos[i]),
                                      (x_nlos[i+1], y_nlos[i+1]),
                                      arrowstyle="->", color="red",
                                      mutation_scale=10, alpha=0.7)
                ax.add_patch(arr)
                nlos_arrows.append(arr)

        # --- LocationAI UPDATE (Blue) ---
        if frame < len(loc_x):
            loc_line.set_data(loc_x[:frame+1], loc_y[:frame+1])
            loc_start.set_offsets([loc_x[0], loc_y[0]])
            loc_end.set_offsets([loc_x[frame], loc_y[frame]])

        return [nlos_scatter, loc_line, loc_start, loc_end] + nlos_arrows

    ani = FuncAnimation(fig, update, frames=max_frames, interval=200, blit=True)

    # Axis setup
    ax.set_xlim([0,65])
    ax.set_ylim([0,28])

    ax.set_title(f"LocationAI (Blue) + NLOS (Red) — Tag {tag_id} ({location_str})")

    # Save the animation
    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)

    print(f"Saved combined video → {output}")

In [ ]:
def animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=None,
        highlight_zones=None,
        output_folder="outputs",
        x_offset=0,
        y_offset=0
    ):

    import matplotlib.image as mpimg
    from matplotlib.animation import FuncAnimation, FFMpegWriter
    from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Final output path
    output = os.path.join(output_folder, f"{tag_id}.mp4")

    # Extract NLOS data
    df_nlos = nlos_df[nlos_df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if df_nlos.empty:
        print(f"No NLOS data for Tag {tag_id}")
        return

    # Extract LocationAI data
    df_loc = voted_df[voted_df["tagId"] == tag_id].sort_values("timestamp").copy()
    if df_loc.empty:
        print(f"No LocationAI data for Tag {tag_id}")
        return

    # Determine location string
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    # Expand centroid values
    xy = df_nlos["Centroid_NLOS"].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
    df_nlos["x"] = xy.apply(lambda v: v[0] + x_offset if isinstance(v, (list, tuple)) else np.nan)
    df_nlos["y"] = xy.apply(lambda v: v[1] + y_offset if isinstance(v, (list, tuple)) else np.nan)

    # Load floor map image
    image = mpimg.imread(map_file) if isinstance(map_file, str) else map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')

        # --- Draw highlighted zones + store centroids ---
    zone_centroids = {}

    # Manual numbering for highlight zones
    manual_zone_numbers = {
        '30558': 1,
        '30509': 2,
        '30553': 4,
        '35182': 3,
        '30514': 6,
        '35181': 7,
        '30516': 8,
        '30512': 5,
        '30518': 9,
        '30533': 13,
        '30520': 10,
        '30524': 11,
        '30525': 12
    }

    for _, row in ground_truth_df.iterrows():
        zone_id = str(row['Zone_id'])  # convert to string to match manual_zone_numbers keys
        x_coords = [row.get(f"x{i+1}") + x_offset for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
        y_coords = [row.get(f"y{i+1}") + y_offset for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]

        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))

            color = "yellow" if not highlight_zones or zone_id in highlight_zones else "none"
            edge_color = "orange" if highlight_zones and zone_id in highlight_zones else "black"

            poly = mplPolygon(coords, closed=True, fill=(color!="none"),
                              facecolor=color, edgecolor=edge_color, alpha=0.3)
            ax.add_patch(poly)

            # Save centroid
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

            # ---- Add manual zone numbering ----
            if zone_id in manual_zone_numbers:
                num = manual_zone_numbers[zone_id]
                cx, cy = zone_centroids[zone_id]

                ax.text(cx, cy, str(num),
                        color="blue",
                        fontsize=12,
                        fontweight="bold",
                        ha="center", va="center")



    # -------- NLOS Scatter (Red) --------
    nlos_scatter = ax.scatter([], [], s=20, color="red")
    nlos_arrows = []

    # -------- LocationAI Path (Blue) --------
    zone_list = list(df_loc["voted_prediction"])
    loc_x = [zone_centroids[z][0] for z in zone_list]
    loc_y = [zone_centroids[z][1] for z in zone_list]

    loc_line, = ax.plot([], [], color="blue", linewidth=2)
    loc_start = ax.scatter([], [], s=100, marker="*", color="blue")
    loc_end   = ax.scatter([], [], s=100, marker="X", color="blue")

    # Animation update function
    max_frames = max(len(df_nlos), len(loc_x))

    def update(frame):
        # --- NLOS UPDATE ---
        x_nlos = df_nlos["x"].values[:frame+1]
        y_nlos = df_nlos["y"].values[:frame+1]

        mask = (~np.isnan(x_nlos)) & (~np.isnan(y_nlos))
        x_nlos = x_nlos[mask]
        y_nlos = y_nlos[mask]

        if len(x_nlos) > 0:
            nlos_scatter.set_offsets(np.column_stack((x_nlos, y_nlos)))
        else:
            nlos_scatter.set_offsets(np.empty((0,2)))

        # Remove arrows from previous frame
        for arr in nlos_arrows:
            arr.remove()
        nlos_arrows.clear()

        # Draw movement arrows
        if len(x_nlos) > 1:
            for i in range(len(x_nlos)-1):
                arr = FancyArrowPatch((x_nlos[i], y_nlos[i]),
                                      (x_nlos[i+1], y_nlos[i+1]),
                                      arrowstyle="->",
                                      color="red",
                                      mutation_scale=10,
                                      alpha=0.7)
                ax.add_patch(arr)
                nlos_arrows.append(arr)

        # --- LocationAI Update ---
        if frame < len(loc_x):
            loc_line.set_data(loc_x[:frame+1], loc_y[:frame+1])
            loc_start.set_offsets([loc_x[0], loc_y[0]])
            loc_end.set_offsets([loc_x[frame], loc_y[frame]])

        return [nlos_scatter, loc_line, loc_start, loc_end] + nlos_arrows

    # Run animation
    ani = FuncAnimation(fig, update, frames=max_frames, interval=200, blit=True)

    # Final configuration
    ax.set_xlim([0,65])
    ax.set_ylim([0,28])
    ax.set_title(f"LocationAI (Blue) + NLOS (Red) — Tag {tag_id} ({location_str})")

    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)

    print(f"Saved combined video → {output}")


In [ ]:
def animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=None,
        highlight_zones=None,
        output_folder="outputs",
        x_offset=0,
        y_offset=0,
        start_coord=None,   # optional start coordinate (x, y)
        end_coord=None,     # optional end coordinate (x, y)
        zone_number_offset=(0.5, 0.5)  # fixed offset for all zone numbers
    ):

    import os
    import ast
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    from matplotlib.animation import FuncAnimation, FFMpegWriter
    from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)
    output = os.path.join(output_folder, f"{tag_id}.mp4")

    # Extract NLOS data
    df_nlos = nlos_df[nlos_df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if df_nlos.empty:
        print(f"No NLOS data for Tag {tag_id}")
        return

    # Extract LocationAI data
    df_loc = voted_df[voted_df["tagId"] == tag_id].sort_values("timestamp").copy()
    if df_loc.empty:
        print(f"No LocationAI data for Tag {tag_id}")
        return

    # Determine location string
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    # Expand centroid values
    xy = df_nlos["Centroid_NLOS"].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
    df_nlos["x"] = xy.apply(lambda v: v[0] + x_offset if isinstance(v, (list, tuple)) else np.nan)
    df_nlos["y"] = xy.apply(lambda v: v[1] + y_offset if isinstance(v, (list, tuple)) else np.nan)

    # Load floor map image
    image = mpimg.imread(map_file) if isinstance(map_file, str) else map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')

    # --- Draw highlighted zones + store centroids ---
    zone_centroids = {}

    # Manual numbering for highlight zones
    manual_zone_numbers = {
        '30558': 1,
        '30509': 2,
        '30553': 4,
        '35182': 3,
        '30514': 6,
        '35181': 7,
        '30516': 8,
        '30512': 5,
        '30518': 9,
        '30533': 13,
        '30520': 10,
        '30524': 11,
        '30525': 12,
        "30530":13,
        "30544": 14,
        "30545":15
    }

    for _, row in ground_truth_df.iterrows():
        zone_id = str(row['Zone_id'])
        x_coords = [row.get(f"x{i+1}") + x_offset for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
        y_coords = [row.get(f"y{i+1}") + y_offset for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]

        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))

            color = "yellow" if not highlight_zones or zone_id in highlight_zones else "none"
            edge_color = "orange" if highlight_zones and zone_id in highlight_zones else "black"

            poly = mplPolygon(coords, closed=True, fill=(color!="none"),
                              facecolor=color, edgecolor=edge_color, alpha=0.3)
            ax.add_patch(poly)

            # Save centroid
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

            # Add manual zone numbering with fixed offset
            if zone_id in manual_zone_numbers:
                num = manual_zone_numbers[zone_id]
                cx, cy = zone_centroids[zone_id]
                dx, dy = zone_number_offset
                ax.text(cx + dx, cy + dy, str(num),
                        color="blue", fontsize=12, fontweight="bold", ha="center", va="center")

    # -------- NLOS Scatter (Red) --------
    nlos_scatter = ax.scatter([], [], s=20, color="red")
    nlos_arrows = []

    # -------- LocationAI Path (Blue) --------
    zone_list = list(df_loc["voted_prediction"])
    loc_x = [zone_centroids[z][0] for z in zone_list]
    loc_y = [zone_centroids[z][1] for z in zone_list]

    loc_line, = ax.plot([], [], color="blue", linewidth=2)
    loc_start = ax.scatter([], [], s=100, marker="*", color="green")
    loc_end   = ax.scatter([], [], s=100, marker="X", color="red")

    # Add custom start/end labels if coordinates provided
    if start_coord:
        ax.scatter(*start_coord, s=100, marker="*", color="green")
        ax.text(start_coord[0], start_coord[1], "Start", color="green", fontsize=12, fontweight="bold",
                ha="left", va="bottom")
    if end_coord:
        ax.scatter(*end_coord, s=100, marker="X", color="red")
        ax.text(end_coord[0], end_coord[1], "End", color="red", fontsize=12, fontweight="bold",
                ha="left", va="bottom")

    # Animation update function
    max_frames = max(len(df_nlos), len(loc_x))

    def update(frame):
        # --- NLOS UPDATE ---
        x_nlos = df_nlos["x"].values[:frame+1]
        y_nlos = df_nlos["y"].values[:frame+1]
        mask = (~np.isnan(x_nlos)) & (~np.isnan(y_nlos))
        x_nlos = x_nlos[mask]
        y_nlos = y_nlos[mask]

        nlos_scatter.set_offsets(np.column_stack((x_nlos, y_nlos)) if len(x_nlos)>0 else np.empty((0,2)))

        # Remove arrows from previous frame
        for arr in nlos_arrows:
            arr.remove()
        nlos_arrows.clear()

        # Draw movement arrows
        if len(x_nlos) > 1:
            for i in range(len(x_nlos)-1):
                arr = FancyArrowPatch((x_nlos[i], y_nlos[i]),
                                      (x_nlos[i+1], y_nlos[i+1]),
                                      arrowstyle="->", color="red",
                                      mutation_scale=10, alpha=0.7)
                ax.add_patch(arr)
                nlos_arrows.append(arr)

        # --- LocationAI Update ---
        if frame < len(loc_x):
            loc_line.set_data(loc_x[:frame+1], loc_y[:frame+1])
            if not start_coord:
                loc_start.set_offsets([loc_x[0], loc_y[0]])
            if not end_coord:
                loc_end.set_offsets([loc_x[frame], loc_y[frame]])

        return [nlos_scatter, loc_line, loc_start, loc_end] + nlos_arrows

    # Run animation
    ani = FuncAnimation(fig, update, frames=max_frames, interval=200, blit=True)

    # Final configuration
    ax.set_xlim([0,65])
    ax.set_ylim([0,28])
    ax.set_title(f"LocationAI (Blue) + NLOS (Red) — Tag {tag_id} ({location_str})")

    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)

    print(f"Saved combined video → {output}")


In [ ]:
highlight_zones=['30558',
 '30509',
 '30553',
 '35182',
 '30514',
 '35181',
 '30516',
 '30512',
 '30518',
 '30533',
 '30520',
 '30524',
 '30525', "30530", "30544", "30545"]

In [ ]:
def animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=None,
        highlight_zones=None,
        output_folder="outputs",
        x_offset=0,
        y_offset=0,
        start_coord=None,   # optional start coordinate (x, y)
        end_coord=None,     # optional end coordinate (x, y)
        zone_number_offset=(0.5, 0.5)  # fixed offset for all zone numbers
    ):

    import os
    import ast
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    from matplotlib.animation import FuncAnimation, FFMpegWriter
    from matplotlib.patches import Polygon as mplPolygon, FancyArrowPatch

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)
    output = os.path.join(output_folder, f"{tag_id}.mp4")

    # Extract NLOS data
    df_nlos = nlos_df[nlos_df["Tag_id"] == tag_id].sort_values("timestamp").copy()
    if df_nlos.empty:
        print(f"No NLOS data for Tag {tag_id}")
        return

    # Extract LocationAI data
    df_loc = voted_df[voted_df["tagId"] == tag_id].sort_values("timestamp").copy()
    if df_loc.empty:
        print(f"No LocationAI data for Tag {tag_id}")
        return

    # Determine location string
    location_str = ""
    if location_map:
        for loc, tag_list in location_map.items():
            if tag_id in tag_list:
                location_str = loc
                break

    # Expand centroid values
    xy = df_nlos["Centroid_NLOS"].apply(lambda v: ast.literal_eval(v) if isinstance(v, str) else v)
    df_nlos["x"] = xy.apply(lambda v: v[0] + x_offset if isinstance(v, (list, tuple)) else np.nan)
    df_nlos["y"] = xy.apply(lambda v: v[1] + y_offset if isinstance(v, (list, tuple)) else np.nan)

    # Load floor map image
    image = mpimg.imread(map_file) if isinstance(map_file, str) else map_file

    fig, ax = plt.subplots(figsize=(14,7))
    ax.imshow(image, extent=[0,65,0,28], aspect='auto')

    # --- Draw highlighted zones + store centroids ---
    zone_centroids = {}

    # Manual numbering for highlight zones
    manual_zone_numbers = {
        '30558': 1,
        '30509': 2,
        '30553': 4,
        '35182': 3,
        '30514': 6,
        '35181': 7,
        '30516': 8,
        '30512': 5,
        '30518': 9,
        '30533': 13,
        '30520': 10,
        '30524': 11,
        '30525': 12,
        "30530":14,
        "30544": 15,
        "30545":16
    }

    for _, row in ground_truth_df.iterrows():
        zone_id = str(row['Zone_id'])
        x_coords = [row.get(f"x{i+1}") + x_offset for i in range(8) if pd.notnull(row.get(f"x{i+1}"))]
        y_coords = [row.get(f"y{i+1}") + y_offset for i in range(8) if pd.notnull(row.get(f"y{i+1}"))]

        if x_coords and y_coords:
            coords = list(zip(x_coords, y_coords))

            color = "yellow" if not highlight_zones or zone_id in highlight_zones else "none"
            edge_color = "orange" if highlight_zones and zone_id in highlight_zones else "black"

            poly = mplPolygon(coords, closed=True, fill=(color!="none"),
                              facecolor=color, edgecolor=edge_color, alpha=0.3)
            ax.add_patch(poly)

            # Save centroid
            zone_centroids[zone_id] = (np.mean(x_coords), np.mean(y_coords))

            # Add manual zone numbering with fixed offset
            if zone_id in manual_zone_numbers:
                num = manual_zone_numbers[zone_id]
                cx, cy = zone_centroids[zone_id]
                dx, dy = zone_number_offset
                ax.text(cx + dx, cy + dy, str(num),
                        color="green", fontsize=12, fontweight="bold", ha="center", va="center")

    # -------- NLOS Scatter (Red) --------
    nlos_scatter = ax.scatter([], [], s=20, color="red")
    nlos_arrows = []

    # -------- LocationAI Path (Blue) --------
    zone_list = list(df_loc["voted_prediction"])
    loc_x = [zone_centroids[z][0] for z in zone_list]
    loc_y = [zone_centroids[z][1] for z in zone_list]

    loc_line, = ax.plot([], [], color="blue", linewidth=2)        # path line
    loc_start = ax.scatter([], [], s=100, marker="*", color="green")  # start
    loc_end   = ax.scatter([], [], s=100, marker="X", color="red")    # end
    loc_current = ax.scatter([], [], s=100, marker="X", color="blue") # moving current position

    # Add custom start/end points if coordinates provided
    if start_coord:
        ax.scatter(*start_coord, s=100, marker="*", color="green")
        ax.text(start_coord[0], start_coord[1], "Start", color="green", fontsize=12, fontweight="bold",
                ha="left", va="bottom")
    if end_coord:
        ax.scatter(*end_coord, s=100, marker="X", color="red")
        ax.text(end_coord[0], end_coord[1], "End", color="red", fontsize=12, fontweight="bold",
                ha="left", va="bottom")

    # Animation update function
    max_frames = max(len(df_nlos), len(loc_x))

    def update(frame):
        # --- NLOS UPDATE ---
        x_nlos = df_nlos["x"].values[:frame+1]
        y_nlos = df_nlos["y"].values[:frame+1]
        mask = (~np.isnan(x_nlos)) & (~np.isnan(y_nlos))
        x_nlos = x_nlos[mask]
        y_nlos = y_nlos[mask]

        nlos_scatter.set_offsets(np.column_stack((x_nlos, y_nlos)) if len(x_nlos)>0 else np.empty((0,2)))

        # Remove arrows from previous frame
        for arr in nlos_arrows:
            arr.remove()
        nlos_arrows.clear()

        # Draw NLOS movement arrows
        if len(x_nlos) > 1:
            for i in range(len(x_nlos)-1):
                arr = FancyArrowPatch((x_nlos[i], y_nlos[i]),
                                      (x_nlos[i+1], y_nlos[i+1]),
                                      arrowstyle="->", color="red",
                                      mutation_scale=10, alpha=0.7)
                ax.add_patch(arr)
                nlos_arrows.append(arr)

        # --- LocationAI Update ---
        if frame < len(loc_x):
            loc_line.set_data(loc_x[:frame+1], loc_y[:frame+1])
            
            if not start_coord:
                loc_start.set_offsets([loc_x[0], loc_y[0]])
            if not end_coord:
                loc_end.set_offsets([loc_x[-1], loc_y[-1]])
            
            # Moving current location marker (blue cross)
            loc_current.set_offsets([loc_x[frame], loc_y[frame]])

        return [nlos_scatter, loc_line, loc_start, loc_end, loc_current] + nlos_arrows

    # Run animation
    ani = FuncAnimation(fig, update, frames=max_frames, interval=200, blit=True)

    # Final configuration
    ax.set_xlim([0,65])
    ax.set_ylim([0,28])
    ax.set_title(f"LocationAI (Blue) + NLOS (Red) — Tag {tag_id} ({location_str})")

    writer = FFMpegWriter(fps=5)
    ani.save(output, writer=writer)
    plt.close(fig)

    print(f"Saved combined video → {output}")


In [ ]:
animate_combined_results(
    final_df,
    voted_df,
    ground_truth_df,
    map_file,
    tag_id='7000140',
    location_map=location_map,
    highlight_zones=highlight_zones,
    output_folder="video_Nov25_data_new",
    start_coord=(5,16),
    end_coord=(10,17.3),
    zone_number_offset=(0, 0.5) 
)

## For all Tags

In [ ]:
def animate_all_tags_combined(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        location_map=location_map,
        highlight_zones=highlight_zones,
        output_folder="video_Nov19_data",
        x_offset=0,
        y_offset=0,
        start_coord=None,   # optional start coordinate (x, y)
        end_coord=None,     # optional end coordinate (x, y)
        zone_number_offset=(0.5, 0.5)
    ):
    import os
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Get all unique tag IDs from NLOS and voted data
    tags_nlos = nlos_df["Tag_id"].unique()
    tags_loc  = voted_df["tagId"].unique()
    all_tags = sorted(set(tags_nlos) | set(tags_loc))  # union of tags
    
    for tag_id in all_tags:
        print(f"Processing Tag {tag_id}...")

        animate_combined_results(
        nlos_df,
        voted_df,
        ground_truth_df,
        map_file,
        tag_id,
        location_map=location_map,
        highlight_zones=highlight_zones,
        output_folder="video_Nov19_data_new",
        x_offset=0,
        y_offset=0,
        start_coord=None,   # optional start coordinate (x, y)
        end_coord=None,     # optional end coordinate (x, y)
        zone_number_offset=(0.5, 0.5)  # fixed offset for all zone numbers
    )
        


In [ ]:
animate_all_tags_combined(
    final_df,
    voted_df,
    ground_truth_df,
    map_file,
    location_map=location_map,
    highlight_zones=highlight_zones,
    output_folder="video_Nov25_data_new",
    start_coord=(5,16),
    end_coord=(10,17.3),
    zone_number_offset=(0, 0.5) 
)
